In [ ]:
root =  "/home/brainmappinglab/Desktop/PROJECTS/Glioblastoma-TractDensity_Surival-Prognosis" # "D:\JoanFR_Sano"
MNI_DIR = "/home/brainmappinglab/Desktop/PROJECTS/MNI_ICBM_2009b_NLIN_ASYM" # "C:/Users/user/Documents/Data/MNI"

In [ ]:
import pandas as pd
import numpy as np
import os
import glob
import matplotlib.pylab as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.cm as cm
import matplotlib.colors as mcolors
from tqdm import tqdm
import nibabel as nib
import nilearn.plotting as plotting

from scipy.stats import pearsonr, linregress, ttest_ind, mannwhitneyu, kendalltau, spearmanr, zscore
from statsmodels.stats.multitest import multipletests, fdrcorrection

from sksurv.nonparametric import kaplan_meier_estimator
from sksurv.compare import compare_survival
from sksurv.linear_model import CoxPHSurvivalAnalysis
from sksurv.metrics import cumulative_dynamic_auc
from sksurv.ensemble import RandomSurvivalForest
from sksurv.preprocessing import OneHotEncoder

from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.cluster import KMeans

import statsmodels.api as sm
import statsmodels.formula.api as smf

In [ ]:
fmt = "pdf"
dpi = 300
stream_th = 0
figs_folder = os.path.join(root,"Relationships-features_anatomy-morphology")
figs_folder_UCSF = os.path.join(figs_folder,"UCSF")
figs_folder_UPENN = os.path.join(figs_folder,"UPENN")
figs_folder_Joint = os.path.join(figs_folder,"Joint")
daysXmonth = 365/12
voxel_size = (0.5**3) * (1/1000) # 0.5 (mm³/voxel) X 0.001 (cm³/mm³)   

os.makedirs(figs_folder, exist_ok=True) 
os.makedirs(figs_folder_UCSF, exist_ok=True) 
os.makedirs(figs_folder_UPENN, exist_ok=True) 
os.makedirs(figs_folder_Joint, exist_ok=True) 

T1 = nib.load(os.path.join(MNI_DIR, "T1_0.5mm_brain.nii.gz"))

def plot_lesions(lesions, mni_img, save_figname=None, title=None, Nslices=20, color_map="Reds", threshold=0, dpi=300):
    fig, ax = plt.subplots(3, 1, figsize=(20, 5), facecolor="k")
    fig.subplots_adjust(hspace=0, left=0, right=1, top=1, bottom=0)
    max_overlap = int(lesions.max())
    lesions = nib.Nifti1Image(lesions, mni_img.affine, mni_img.header)

    display_x = plotting.plot_anat(
        mni_img, 
        axes=ax[0], 
        display_mode="x", 
        cut_coords=np.linspace(-80,80,Nslices), 
        draw_cross=False, 
        black_bg=True,
        annotate=False
    )
    display_x.add_overlay(
        lesions,
        cmap = color_map,
        colorbar=True,
        cbar_vmin=threshold,
        cbar_vmax=max_overlap,
        cbar_tick_format="%i",
        threshold=threshold
    )

    display_y = plotting.plot_anat(
        mni_img, 
        axes=ax[1], 
        display_mode="y", 
        cut_coords=np.linspace(-80,80,Nslices), 
        draw_cross=False, 
        black_bg=True,
        annotate=False
    )
    display_y.add_overlay(
        lesions,
        cmap = color_map,
        colorbar=False,
        cbar_vmin=threshold,
        cbar_vmax=max_overlap,
        cbar_tick_format="%i",
        threshold=threshold
    )

    display_z = plotting.plot_anat(
        mni_img, 
        axes=ax[2], 
        display_mode="z", 
        cut_coords=np.linspace(-80,80,Nslices), 
        draw_cross=False, 
        black_bg=True,
        annotate=False
    )
    display_z.add_overlay(
        lesions,
        cmap = color_map,
        colorbar=False,
        cbar_vmin=threshold,
        cbar_vmax=max_overlap,
        cbar_tick_format="%i",
        threshold=threshold
    )
    fig.suptitle(title, fontweight='bold', color='white', fontsize=20)
    if save_figname is not None:
        fig.savefig(save_figname, dpi=dpi, format='pdf')

### UCSF

In [ ]:
root_ucsf = os.path.join(root,"Glioblastoma_UCSF-PDGM_v3-20230111")
TDstats_ucsf = pd.read_csv(os.path.join(root_ucsf, f"TDMaps_Grade-IV/demographics-TDMaps_streamTH-{stream_th}.csv"))
morphology_ucsf = pd.read_csv(os.path.join(root_ucsf, f"TDMaps_Grade-IV/morphology-tissues.csv"))

TDstats_ucsf = TDstats_ucsf.loc[TDstats_ucsf["Final pathologic diagnosis (WHO 2021)"]=="Glioblastoma  IDH-wildtype"] 
TDstats_ucsf = TDstats_ucsf.loc[TDstats_ucsf["OS"].fillna('unknown')!='unknown']
morphology_ucsf = morphology_ucsf.loc[morphology_ucsf["Final pathologic diagnosis (WHO 2021)"]=="Glioblastoma  IDH-wildtype"] 
morphology_ucsf = morphology_ucsf.loc[morphology_ucsf["OS"].fillna('unknown')!='unknown']

common_cols = ['ID', 'Sex', 'Age at MRI', 'WHO CNS Grade', 
       'Final pathologic diagnosis (WHO 2021)', 'MGMT status', 'MGMT index',
       '1p/19q', 'IDH', '1-dead 0-alive', 'OS', 'EOR',
       'Biopsy prior to imaging', 'BraTS21 ID', 'BraTS21 Segmentation Cohort',
       'BraTS21 MGMT Cohort', '# Labels'
]
ucsf = pd.merge(TDstats_ucsf, morphology_ucsf, on=common_cols)

print(ucsf["1-dead 0-alive"].value_counts().sum())
censored = (ucsf["1-dead 0-alive"]==0).sum()
all_ucsf = ucsf["1-dead 0-alive"].value_counts().sum()
print(f"Precentage of censoring: {round(100*censored/all_ucsf,2)}%")

### UPENN

In [ ]:
root_upenn = os.path.join(root,"Glioblastoma_UPENN-GBM_v2-20221024")
TDstats_upenn = pd.read_csv(os.path.join(root_upenn, f"TDMaps_IDH1-WT/demographics-TDMaps_streamTH-{stream_th}.csv"))
morphology_upenn = pd.read_csv(os.path.join(root_upenn, f"TDMaps_IDH1-WT/morphology-tissues.csv"))
    
common_cols = ["ID", "Gender", "Age_at_scan_years", "Survival_from_surgery_days_UPDATED", "Survival_Status", "MGMT", "KPS", "GTR_over90percent"]
upenn = pd.merge(TDstats_upenn, morphology_upenn, on=common_cols)

print(upenn["Survival_Status"].value_counts().sum())
censored = (upenn["Survival_Status"]==0).sum()
all_upenn = upenn["Survival_Status"].value_counts().sum()
print(f"Precentage of censoring: {round(100*censored/all_upenn,2)}%")

### Quantifying the effect of site

#### Generalized Linear Models for the features

In [ ]:
z_transform = 0

# UCSF
df_ucsf = pd.DataFrame({
    "ID": ucsf["ID"].values,
    "sex": ucsf["Sex"].values,
    "age": ucsf["Age at MRI"].values,
    "site": np.ones(len(ucsf), dtype=int),  # UCSF = 1
    "volume": ucsf["Whole tumor size (voxels)"].values * voxel_size,
    "ltdi": ucsf["Whole lesion TDMap"].values,
    "tdi": ucsf["Whole TDMap"].values,
    "OS": ucsf["OS"].values,
    "status": ucsf["1-dead 0-alive"]
})

# UPENN
df_upenn = pd.DataFrame({
    "ID": upenn["ID"].values,
    "sex": upenn["Gender"].values,
    "age": upenn["Age_at_scan_years"].values,
    "site": np.ones(len(upenn), dtype=int) + 1,  # UPENN = 2
    "volume": upenn["Whole tumor size (voxels)"].values * voxel_size,
    "ltdi": upenn["Whole lesion TDMap"].values,
    "tdi": upenn["Whole TDMap"].values,
    "OS": upenn["Survival_from_surgery_days_UPDATED"],
    "status": upenn["Survival_Status"]
})

DATA = pd.concat([df_ucsf, df_upenn], axis=0, ignore_index=True)

if z_transform == 1:
    for col in ["age", "volume", "ltdi", "tdi"]:
        DATA[col] = zscore(DATA[col])

GLM = smf.glm(formula="ltdi ~ sex + age + site", data=DATA)
LTDI_MODEL = GLM.fit()
LTDI_res = DATA["ltdi"] - LTDI_MODEL.predict() + LTDI_MODEL.params["Intercept"]
LTDI_SUMMARY = LTDI_MODEL.summary()

GLM = smf.glm(formula="tdi ~ sex + age + site", data=DATA)
TDI_MODEL = GLM.fit()
TDI_res = DATA["ltdi"] - LTDI_MODEL.predict() + LTDI_MODEL.params["Intercept"]
TDI_SUMMARY = TDI_MODEL.summary()

GLM = smf.glm(formula="volume ~ sex + age + site", data=DATA)
VOLUME_MODEL = GLM.fit()
VOLUME_res = DATA["ltdi"] - LTDI_MODEL.predict() + LTDI_MODEL.params["Intercept"]
VOLUME_SUMMARY = VOLUME_MODEL.summary()

DATA.head()

In [ ]:
LTDI_SUMMARY

In [ ]:
TDI_SUMMARY

In [ ]:
VOLUME_SUMMARY

#### Survival differences in site

In [ ]:
months = range(0,110,10)

# Pre-correction
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(10,5))
colors = ["tab:green", "tab:purple"]
OS_STATS = []
GROUP_STATS = []
nums = np.empty(shape=(2,), dtype=object)
for i, cohort in enumerate([1, 2]):
    diag = DATA[DATA["site"]==cohort]
    mask = ~np.isnan(diag["status"]) & ~np.isnan(diag["OS"])
    diag = diag[mask]
    nums[i] = []
    for t in months:
        nums[i].append((diag["OS"]>=(t*daysXmonth)).sum())
    
    time, survival_prob, conf_int = kaplan_meier_estimator(
                diag["status"]==1, diag["OS"], conf_type="log-log"
            )
    ax1.step(time/daysXmonth, survival_prob, where="post", label=f"Male", color=colors[i])
    ax1.fill_between(time/daysXmonth, conf_int[0], conf_int[1], alpha=0.15, step="post", color=colors[i])
    for t in diag.loc[diag["status"]==0, "OS"].values: # Censoring times
        ax1.plot(time[time==t]/daysXmonth, survival_prob[time==t], "|", color=colors[i])

    OS_STATS.extend([(st, ovs) for st, ovs in zip(diag["status"]==1, diag["OS"].values)])
    GROUP_STATS.extend([i+1 for ovs in diag["OS"].values])

OS_STATS = np.array(OS_STATS, dtype=[('event', 'bool'),('time', 'float')])
chisquared, p_val, stats, covariance = compare_survival(OS_STATS, GROUP_STATS, return_stats=True)
ax1.text(0.70, 0.90, r"$\chi^2 =$"+f"{round(chisquared,4)}, \np = {round(p_val,4)}", transform=ax1.transAxes, 
                    fontsize=10, verticalalignment='top', bbox=dict(boxstyle="round", alpha=0.1), color="red" if p_val<=0.05 else "black")        

for i,t in enumerate(months):
    ax1.text(t-2, -0.07, f"{nums[0][i]}", transform=ax1.transData, fontsize=11, verticalalignment='top', color=colors[0]) 
    ax1.text(t-2, -0.13, f"{nums[1][i]}", transform=ax1.transData, fontsize=11, verticalalignment='top', color=colors[1]) 
ax1.text(-2, -0.01, "No. at risk", transform=ax1.transData, fontsize=11, verticalalignment='top', color="black", fontweight='bold') 
ax1.hlines(0,-5,months[-1]+5, color="black", linewidth=.5)
ax1.set_ylim([-.2,1])
ax1.set_xlim([-5,75])
ax1.set_xticks(range(0,months[-1]+10,10))
ax1.set_xticklabels(range(0,months[-1]+10,10))
ax1.set_yticks([0,0.2,0.4,0.6,0.8,1])
ax1.set_yticklabels([0,0.2,0.4,0.6,0.8,1])
ax1.spines['left'].set_bounds(0,1)
ax1.spines['bottom'].set_bounds(0,months[-1])
ax1.set_xlabel("Time (months)", fontsize=12)
ax1.set_ylabel("Overall survival", fontsize=12)
ax1.spines[["top", "right"]].set_visible(False)

# Statistical insurance of the effect
n_perms = 1000
Cmodel = CoxPHSurvivalAnalysis(n_iter=200)
Cmodel.fit(DATA["site"].values.reshape(-1,1), OS_STATS)
c_index = Cmodel.score(DATA["site"].values.reshape(-1,1), OS_STATS)

pop = []
for perm_i in tqdm(range(n_perms)):
    perm_OS_STATS = np.random.permutation(OS_STATS)
    p_Cmodel = CoxPHSurvivalAnalysis()
    p_Cmodel.fit(DATA["site"].values.reshape(-1, 1), perm_OS_STATS)
    pop.append(p_Cmodel.score(DATA["site"].values.reshape(-1, 1), perm_OS_STATS))
p_value = np.mean(np.array(pop) >= c_index)
print(f"C-index = {c_index} (p={p_value})")
print(f"Log HR = {Cmodel.coef_}")

# Post-correction
colors = ["tab:green", "tab:purple"]
OS_STATS_deSITE = []
GROUP_STATS = []
nums = np.empty(shape=(2,), dtype=object)
for i, cohort in enumerate([1, 2]):
    diag = DATA[DATA["site"]==cohort]
    mask = ~np.isnan(diag["status"]) & ~np.isnan(diag["OS"])
    diag = diag[mask]
    diag["OS"] = diag["OS"] * np.exp(Cmodel.coef_[0]*cohort)
    nums[i] = []
    for t in months:
        nums[i].append((diag["OS"]>=(t*daysXmonth)).sum())
    
    time, survival_prob, conf_int = kaplan_meier_estimator(
                diag["status"]==1, diag["OS"], conf_type="log-log"
            )
    ax2.step(time/daysXmonth, survival_prob, where="post", label=f"Male", color=colors[i])
    ax2.fill_between(time/daysXmonth, conf_int[0], conf_int[1], alpha=0.15, step="post", color=colors[i])
    for t in diag.loc[diag["status"]==0, "OS"].values: # Censoring times
        ax2.plot(time[time==t]/daysXmonth, survival_prob[time==t], "|", color=colors[i])

    OS_STATS_deSITE.extend([(st, ovs) for st, ovs in zip(diag["status"]==1, diag["OS"].values)])
    GROUP_STATS.extend([i+1 for ovs in diag["OS"].values])

OS_STATS_deSITE = np.array(OS_STATS_deSITE, dtype=[('event', 'bool'),('time', 'float')])
chisquared, p_val, stats, covariance = compare_survival(OS_STATS_deSITE, GROUP_STATS, return_stats=True)
ax2.text(0.70, 0.90, r"$\chi^2 =$"+f"{round(chisquared,4)}, \np = {round(p_val,4)}", transform=ax2.transAxes, 
                    fontsize=10, verticalalignment='top', bbox=dict(boxstyle="round", alpha=0.1), color="red" if p_val<=0.05 else "black")        

for i,t in enumerate(months):
    ax2.text(t-2, -0.07, f"{nums[0][i]}", transform=ax2.transData, fontsize=11, verticalalignment='top', color=colors[0]) 
    ax2.text(t-2, -0.13, f"{nums[1][i]}", transform=ax2.transData, fontsize=11, verticalalignment='top', color=colors[1]) 
ax2.text(-2, -0.01, "No. at risk", transform=ax2.transData, fontsize=11, verticalalignment='top', color="black", fontweight='bold') 
ax2.hlines(0,-5,months[-1]+5, color="black", linewidth=.5)
ax2.set_ylim([-.2,1])
ax2.set_xlim([-5,75])
ax2.set_xticks(range(0,months[-1]+10,10))
ax2.set_xticklabels(range(0,months[-1]+10,10))
ax2.set_yticks([0,0.2,0.4,0.6,0.8,1])
ax2.set_yticklabels([0,0.2,0.4,0.6,0.8,1])
ax2.spines['left'].set_bounds(0,1)
ax2.spines['bottom'].set_bounds(0,months[-1])
ax2.set_xlabel("Time (months)", fontsize=12)
ax2.set_ylabel("Overall survival", fontsize=12)
ax2.spines[["top", "right"]].set_visible(False)

fig.tight_layout()
fig.savefig(os.path.join(figs_folder, f"Site-effects_Survival-times.{fmt}"), dpi=dpi, format=fmt)

### Site differences for the 3 features

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(6,5))

volumes_ucsf = DATA.loc[DATA["site"]==1, "volume"]
volumes_upenn = DATA.loc[DATA["site"]==2, "volume"]

median_ucsf = np.median(volumes_ucsf)
q1_ucsf = np.percentile(volumes_ucsf, 25)
q3_ucsf = np.percentile(volumes_ucsf, 75)

median_upenn = np.median(volumes_upenn)
q1_upenn = np.percentile(volumes_upenn, 25)
q3_upenn = np.percentile(volumes_upenn, 75)

hist_ucsf = ax.hist(volumes_ucsf, histtype="step", density=False, cumulative=False, bins=50, 
                    color="tab:green", linewidth=3, label="UCSF cohort")
hist_upenn = ax.hist(volumes_upenn, histtype="step", density=False, cumulative=False, bins=50, 
                     color="tab:purple", linewidth=3, 
                     weights=-np.ones_like(volumes_upenn), label="UPENN cohort")

ax.hlines(0, 0, 354, color="gray", linewidth=1, linestyle="--")

ax.vlines(median_ucsf, 0, 25, color="black", linestyle="-", linewidth=5)
ax.fill_between([q1_ucsf, q3_ucsf], [0, 0], [25, 25], alpha=0.25, facecolor="gray")

ax.vlines(median_upenn, -26, 0, color="black", linestyle="-", linewidth=5)
ax.fill_between([q1_upenn, q3_upenn], [-26, -26], [0, 0], alpha=0.25, color="gray")

U, p = mannwhitneyu(volumes_ucsf, volumes_upenn, alternative="two-sided")
ax.text(0.75, 0.15, f"t = {round(U,4)} \np = {round(p,4)}", transform=ax.transAxes, 
                    fontsize=10, verticalalignment='top', bbox=dict(boxstyle="round", alpha=0.1), color="red" if p<=0.05 else "black")   

# Customize the axis appearance
ax.spines[["top", "right"]].set_visible(False)
ax.set_xlim([-10, 370])
ax.set_ylim([-30,30])
ax.set_yticks(range(-30,35,10))
ax.set_yticklabels([30,20,10,0,10,20,30])
ax.spines['bottom'].set_bounds(0, 350)
ax.set_xlabel("Tumor volume ("+r'$cm^3$'+")", fontsize=12)
ax.set_ylabel("Counts", fontsize=12)
ax.legend(frameon=False)

fig.tight_layout()
fig.savefig(os.path.join(figs_folder, f"Site-effects_Tumor-volume.{fmt}"), dpi=dpi, format=fmt)

U, p = mannwhitneyu(volumes_ucsf, volumes_upenn, alternative="two-sided")
print(f"UCSF cohort median tumor size: {volumes_ucsf.median()} [{q1_ucsf}, {q3_ucsf}] (cm3)")
print(f"UCSF cohort median tumor size: {volumes_upenn.median()} [{q1_upenn}, {q3_upenn}] (cm3)")
print(f"U = {U}, p = {p}")

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(6,5))

ltdi_ucsf = DATA.loc[DATA["site"]==1, "ltdi"]
ltdi_upenn = DATA.loc[DATA["site"]==2, "ltdi"]

median_ucsf = np.median(ltdi_ucsf)
q1_ucsf = np.percentile(ltdi_ucsf, 25)
q3_ucsf = np.percentile(ltdi_ucsf, 75)

median_upenn = np.median(ltdi_upenn)
q1_upenn = np.percentile(ltdi_upenn, 25)
q3_upenn = np.percentile(ltdi_upenn, 75)

hist_ucsf = ax.hist(ltdi_ucsf, histtype="step", density=False, cumulative=False, bins=50, 
                    color="tab:green", linewidth=3, label="UCSF cohort")
hist_upenn = ax.hist(ltdi_upenn, histtype="step", density=False, cumulative=False, bins=50, 
                     color="tab:purple", linewidth=3, 
                     weights=-np.ones_like(ltdi_upenn), label="UPENN cohort")

ax.hlines(0, 0, 15, color="gray", linewidth=1, linestyle="--")

ax.vlines(median_ucsf, 0, 25, color="black", linestyle="-", linewidth=5)
ax.fill_between([q1_ucsf, q3_ucsf], [0, 0], [25, 25], alpha=0.25, facecolor="gray")

ax.vlines(median_upenn, -30, 0, color="black", linestyle="-", linewidth=5)
ax.fill_between([q1_upenn, q3_upenn], [-30, -30], [0, 0], alpha=0.25, color="gray")

U, p = mannwhitneyu(ltdi_ucsf, ltdi_upenn, alternative="two-sided")
ax.text(0.80, 0.10, f"U = {round(U,4)} \np = {round(p,4)}", transform=ax.transAxes, 
                    fontsize=10, verticalalignment='top', bbox=dict(boxstyle="round", alpha=0.1), color="red" if p<=0.05 else "black")   

# Customize the axis appearance
ax.spines[["top", "right"]].set_visible(False)
ax.set_xlim([-1, 14])
ax.set_ylim([-35,30])
ax.set_yticks(range(-30,35,10))
ax.set_yticklabels([30,20,10,0,10,20,30])
ax.spines['bottom'].set_bounds(0, 14)
ax.set_xlabel("L-TDI", fontsize=12)
ax.set_ylabel("Counts", fontsize=12)
ax.legend(frameon=False)

fig.tight_layout()
fig.savefig(os.path.join(figs_folder, f"Site-effects_LTDI.{fmt}"), dpi=dpi, format=fmt)

print(f"UCSF cohort median tumor size: {ltdi_ucsf.median()} [{q1_ucsf}, {q3_ucsf}] (cm3)")
print(f"UCSF cohort median tumor size: {ltdi_upenn.median()} [{q1_upenn}, {q3_upenn}] (cm3)")
print(f"U = {U}, p = {p}")

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(6,5))

tdi_ucsf = DATA.loc[DATA["site"]==1, "tdi"]
tdi_upenn = DATA.loc[DATA["site"]==2, "tdi"]

median_ucsf = np.median(tdi_ucsf)
q1_ucsf = np.percentile(tdi_ucsf, 25)
q3_ucsf = np.percentile(tdi_ucsf, 75)

median_upenn = np.median(tdi_upenn)
q1_upenn = np.percentile(tdi_upenn, 25)
q3_upenn = np.percentile(tdi_upenn, 75)

hist_ucsf = ax.hist(tdi_ucsf, histtype="step", density=False, cumulative=False, bins=50, 
                    color="tab:green", linewidth=3, label="UCSF cohort")
hist_upenn = ax.hist(tdi_upenn, histtype="step", density=False, cumulative=False, bins=50, 
                     color="tab:purple", linewidth=3, 
                     weights=-np.ones_like(tdi_upenn), label="UPENN cohort")

ax.hlines(0, 0, 300, color="gray", linewidth=1, linestyle="--")

ax.vlines(median_ucsf, 0, 30, color="black", linestyle="-", linewidth=5)
ax.fill_between([q1_ucsf, q3_ucsf], [0, 0], [30, 30], alpha=0.25, facecolor="gray")

ax.vlines(median_upenn, -35, 0, color="black", linestyle="-", linewidth=5)
ax.fill_between([q1_upenn, q3_upenn], [-35, -35], [0, 0], alpha=0.25, color="gray")

tscore, p = ttest_ind(tdi_ucsf, tdi_upenn, alternative="two-sided")
ax.text(0.80, 0.10, f"t = {round(tscore,4)} \np = {round(p,4)}", transform=ax.transAxes, 
                    fontsize=10, verticalalignment='top', bbox=dict(boxstyle="round", alpha=0.1), color="red" if p<=0.05 else "black")    

# Customize the axis appearance
ax.spines[["top", "right"]].set_visible(False)
ax.set_xlim([-10, 300])
ax.set_ylim([-40,35])
ax.set_yticks(range(-40,40,10))
ax.set_yticklabels([40,30,20,10,0,10,20,30])
ax.spines['bottom'].set_bounds(0, 300)
ax.set_xlabel("TDI", fontsize=12)
ax.set_ylabel("Counts", fontsize=12)
ax.legend(frameon=False)

fig.tight_layout()
fig.savefig(os.path.join(figs_folder, f"Site-effects_TDI.{fmt}"), dpi=dpi, format=fmt)

print(f"UCSF cohort median tumor size: {tdi_ucsf.median()} [{q1_ucsf}, {q3_ucsf}] (cm3)")
print(f"UCSF cohort median tumor size: {tdi_upenn.median()} [{q1_upenn}, {q3_upenn}] (cm3)")
print(f"t = {tscore}, p = {p}")

### Relationship between features

In [ ]:
colors_scatters = ["tab:green", "tab:purple"]
colors_clusters = {
    1: ['#d95f02', '#1b9e77', '#7570b3'], # Cluster 0: Orange, Cluster 1: Teal, Cluster 2: Purple 
    2: ['#7570b3', '#1b9e77', '#d95f02'] # Cluster 0: Purple, Cluster 1: Teal, Cluster 2: Orange
}
titles = {
    1: f"UCSF Cohort (N={all_ucsf})",
    2: f"UPENN Cohort (N={all_upenn})"
}

cohorts = [1, 2]

In [ ]:
saves = {
    1: os.path.join(figs_folder_UCSF, f"Scatter-distributions_LTDI-TDI-Volume_UCSF.{fmt}"),
    2: os.path.join(figs_folder_UPENN, f"Scatter-distributions_LTDI-TDI-Volume_UPENN.{fmt}")
}
for cohort in cohorts:
    fig, ax = plt.subplots(1, 1, figsize=(5, 4))

    sc = ax.scatter(DATA.loc[DATA["site"] == cohort, "tdi"], 
                        DATA.loc[DATA["site"] == cohort, "ltdi"], 
                        c=DATA.loc[DATA["site"] == cohort, "volume"], 
                        cmap="viridis", marker='o', alpha=1, s=55)
    ax.spines[["top","right"]].set_visible(False)
    ax.set_xlim([-5,320])
    ax.set_ylim([-.1,12])
    ax.spines['bottom'].set_bounds(0, 300)
    ax.spines['left'].set_bounds(0, 12)
    ax.set_xlabel("Tract density index (TDI)", fontsize=12)
    ax.set_ylabel("Lesion-tract density index (L-TDI)", fontsize=12)
    ax.set_title(titles[cohort], fontweight="bold")

    corr, pval = pearsonr(DATA.loc[DATA["site"]==cohort, "tdi"], DATA.loc[DATA["site"]==cohort, "ltdi"])
    ax.text(
        0.05, 0.95, 
        r'$\rho$'+f" = {corr:.3f}\np = {pval:.3f}", 
        transform=ax.transAxes, verticalalignment='top', bbox=dict(boxstyle="round", alpha=0.1), 
        color='red' if pval < 0.05 else 'black', 
        fontsize=12
    )

    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    cbar = fig.colorbar(sc, cax=cax)
    cbar.set_label("Tumor volume ("+r'$cm^3$'+")", fontsize=12)

    fig.tight_layout()
    fig.savefig(saves[cohort], dpi=dpi, format=fmt)

In [ ]:
saves = {
    1: os.path.join(figs_folder_UCSF, f"Scatter-distributions_UCSF-VlmTDI.{fmt}"),
    2: os.path.join(figs_folder_UPENN, f"Scatter-distributions_UPENN-VlmTDI.{fmt}")
}
KMEANS_cohorts = dict()

for cohort in cohorts:
    kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
    kmeans.fit_predict(DATA.loc[DATA["site"]==cohort, ["volume", "tdi"]])
    KMEANS_cohorts[cohort] = kmeans
    
    fig, ax = plt.subplots(2, 2, figsize=(8,8))

    ax[0,0].scatter(DATA.loc[DATA["site"]==cohort, "age"], DATA.loc[DATA["site"]==cohort, "ltdi"], s=10, color=colors_scatters[cohort-1])
    ax[0,0].spines[["top","right","bottom"]].set_visible(False)
    ax[0,0].set_xlim([15,100])
    ax[0,0].set_ylim([0,15])
    ax[0,0].set_xticks([])
    ax[0,0].set_ylabel("Lesion-tract density index", fontsize=12)
    corr, pval = pearsonr(DATA.loc[DATA["site"]==cohort, "age"], DATA.loc[DATA["site"]==cohort, "ltdi"])
    color = 'red' if pval < 0.05 else 'black'
    ax[0,0].text(0.05, 0.9, r'$\rho$'+f" = {corr:.2f}\np = {pval:.3f}", transform=ax[0,0].transAxes, verticalalignment='top', bbox=dict(boxstyle="round", alpha=0.1), color=color, fontsize=12)

    ax[1,0].scatter(DATA.loc[DATA["site"]==cohort, "age"], DATA.loc[DATA["site"]==cohort, "tdi"], s=10, color=colors_scatters[cohort-1])
    ax[1,0].spines[["top","right"]].set_visible(False)
    ax[1,0].set_xlim([15,100])
    ax[1,0].set_ylim([0,300])
    ax[1,0].set_ylabel("Tract density index", fontsize=12)
    ax[1,0].set_xlabel("Age at MRI (years)", fontsize=12)
    corr, pval = pearsonr(DATA.loc[DATA["site"]==cohort, "age"], DATA.loc[DATA["site"]==cohort, "tdi"])
    color = 'red' if pval < 0.05 else 'black'
    ax[1,0].text(0.05, 0.9, r'$\rho$'+f" = {corr:.2f}\np = {pval:.3f}", transform=ax[1,0].transAxes, verticalalignment='top', bbox=dict(boxstyle="round", alpha=0.1), color=color, fontsize=12)

    ax[0,1].scatter(DATA.loc[DATA["site"]==cohort, "volume"], DATA.loc[DATA["site"]==cohort, "ltdi"], s=10, color=colors_scatters[cohort-1])
    ax[0,1].spines[["top","right","bottom"]].set_visible(False)
    ax[0,1].set_xlim([0,320])
    ax[0,1].set_ylim([0,15])
    ax[0,1].set_xticks([])
    ax[0,1].set_ylabel("Lesion-tract density index", fontsize=12)
    corr, pval = pearsonr(DATA.loc[DATA["site"]==cohort, "volume"], DATA.loc[DATA["site"]==cohort, "ltdi"])
    color = 'red' if pval < 0.05 else 'black'
    ax[0,1].text(0.05, 0.9, r'$\rho$'+f" = {corr:.2f}\np = {pval:.3f}", transform=ax[0,1].transAxes, verticalalignment='top', bbox=dict(boxstyle="round", alpha=0.1), color=color, fontsize=12)

    ax[1,1].scatter(DATA.loc[DATA["site"]==cohort, "volume"], DATA.loc[DATA["site"]==cohort, "tdi"], s=10, c=[colors_clusters[cohort][label] for label in kmeans.labels_], marker='o', alpha=0.25)
    for i in range(kmeans.n_clusters):
        ax[1,1].scatter(kmeans.cluster_centers_[i, 0], kmeans.cluster_centers_[i, 1], c=colors_clusters[cohort][i], marker='x', s=100, linewidth=3)
    ax[1,1].spines[["top","right"]].set_visible(False)
    ax[1,1].set_xlim([0,320])
    ax[1,1].set_ylim([0,300])
    ax[1,1].set_xlabel("Tumor volume ("+r'$cm^3$'+")", fontsize=12)
    ax[1,1].set_ylabel("Tract density index", fontsize=12)
    corr, pval = pearsonr(DATA.loc[DATA["site"]==cohort, "volume"], DATA.loc[DATA["site"]==cohort, "tdi"])
    color = 'red' if pval < 0.05 else 'black' 
    ax[1,1].text(0.70, 0.9, r'$\rho$'+f" = {corr:.2f}\np = {pval:.3f}", transform=ax[1,1].transAxes, verticalalignment='top', bbox=dict(boxstyle="round", alpha=0.1), color=color, fontsize=12)

    fig.suptitle(titles[cohort], fontweight="bold")
    fig.tight_layout()
    fig.savefig(saves[cohort], dpi=dpi, format=fmt)

In [ ]:
cohort = 1
X = DATA.loc[DATA["site"] == cohort, ["tdi", "ltdi", "volume"]].dropna().values  

# Apply PCA
pca = PCA(n_components=3)  
X_pca = pca.fit_transform(X)
explained_variance = pca.explained_variance_ratio_
y_1 = DATA.loc[DATA["site"] == cohort, "ltdi"]
pca_model = LinearRegression(fit_intercept=True)
pca_model.fit(X_pca[:,:-1], y_1)
y_pred_1 = pca_model.predict(X_pca[:,:-1])

fig, ax = plt.subplots(1, 4, figsize=(16,5), constrained_layout=True)

sc0 = ax[0].scatter(X_pca[:, 0], X_pca[:, 1], c=DATA.loc[DATA["site"] == cohort, "volume"], cmap="viridis", marker='o', alpha=0.7)
ax[0].spines[["top", "right"]].set_visible(False)
ax[0].set_xlim([-150,250])
ax[0].set_ylim([-150,250])
ax[0].set_xlabel(f"PC1 ({explained_variance[0]*100:.1f}%)")
ax[0].set_ylabel(f"PC2 ({explained_variance[1]*100:.1f}%)")

sc1 = ax[1].scatter(X_pca[:, 0], X_pca[:, 1], c=DATA.loc[DATA["site"] == cohort, "tdi"], cmap="viridis", marker='o', alpha=0.7)
ax[1].spines[["top", "right", "left"]].set_visible(False)
ax[1].set_xlim([-150,250])
ax[1].set_ylim([-150,250])
ax[1].set_yticks([])
ax[1].set_xlabel(f"PC1 ({explained_variance[0]*100:.1f}%)")

# Plot isocontours (lines of constant L-TDI)
norm = mcolors.Normalize(vmin=y_1.min(), vmax=y_1.max())  
cmap = cm.viridis  
ltdi_values = [2, 4, 6, 8]
x_ranges = [np.array([-120, -10]), np.array([-90, 70]), np.array([-60, 150]), np.array([10, 220])]
for iso, x_range in zip(ltdi_values, x_ranges):
    color = cmap(norm(iso))  
    y_range = (iso - pca_model.intercept_ - pca_model.coef_[0] * x_range) / pca_model.coef_[1]
    ax[2].plot(x_range, y_range, linewidth=8, color='black', alpha=1, zorder=1)  
    ax[2].plot(x_range, y_range, linewidth=4, color=color, alpha=0.75, zorder=2)
sc2 = ax[2].scatter(X_pca[:, 0], X_pca[:, 1], c=y_1, cmap="viridis", marker='o', alpha=0.5)
arrow_length = 200
arrow_dx = arrow_length
arrow_dy = arrow_length * pca_model.coef_[1] / pca_model.coef_[0]
ax[2].arrow(
    -0, 
    -0, 
    arrow_dx, 
    arrow_dy, 
    color="black", 
    head_width=5, 
    head_length=5, 
    linewidth=2.5, 
    alpha=0.75, 
    label=f"Rotation of {round(np.rad2deg(np.arctan(pca_model.coef_[1] / pca_model.coef_[0])),2)}º"
)
ax[2].spines[["top", "right"]].set_visible(False)
ax[2].set_xlim([-150, 250])
ax[2].set_ylim([-150, 250])
ax[2].set_xlabel(f"PC1 ({explained_variance[0]*100:.1f}%)")
ax[2].set_ylabel(f"PC2 ({explained_variance[1]*100:.1f}%)")
ax[2].legend(frameon=False, loc="upper right")

sc3 = ax[3].scatter(y_pred_1, y_1, marker='o', alpha=0.7, c='tab:green', s=10)
ax[3].plot([0, max(y_pred_1)], [0, max(y_pred_1)], 'k-', lw=2, alpha=0.5)
ax[3].text(0.10, 0.90, r'$R^2$'+f" = {round(pca_model.score(X_pca[:,:-1],y_1),2)}", transform=ax[3].transAxes, 
                    fontsize=10, verticalalignment='top', bbox=dict(boxstyle="round", alpha=0.1), color="black")  
ax[3].spines[["top", "right"]].set_visible(False)
ax[3].set_xlim([-1,10])
ax[3].set_ylim([-1,14])
ax[3].set_xlabel(f"L-TDI ~ {round(pca_model.coef_[0],3)}·PC1 + {round(pca_model.coef_[1],3)}·PC2 + {round(pca_model.intercept_,3)}")
ax[3].set_ylabel("L-TDI")

# Add colorbars below each subplot
cbar0 = fig.colorbar(sc0, ax=ax[0], orientation="horizontal", fraction=0.05, pad=0.02)
cbar0.set_label("Tumor volume (" + r'$cm^3$' + ")")

cbar1 = fig.colorbar(sc1, ax=ax[1], orientation="horizontal", fraction=0.05, pad=0.02)
cbar1.set_label("TDI (a.u.)")

cbar2 = fig.colorbar(sc2, ax=ax[2], orientation="horizontal", fraction=0.05, pad=0.02)
cbar2.set_label("L-TDI (a.u.)")

fig.suptitle(f"UCSF Cohort (N={all_ucsf})", fontweight="bold")
fig.savefig(os.path.join(figs_folder_UCSF, f"Scatter-distributions_LTDI-TDI-Volume_UCSF-PCA.{fmt}"), dpi=dpi, format=fmt)

In [ ]:
cohort = 2
X = DATA.loc[DATA["site"] == cohort, ["tdi", "ltdi", "volume"]].dropna().values  

# Apply PCA
pca = PCA(n_components=3)  
X_pca = pca.fit_transform(X)
explained_variance = pca.explained_variance_ratio_
y_2 = DATA.loc[DATA["site"] == cohort, "ltdi"]
pca_model = LinearRegression()
pca_model.fit(X_pca[:,:-1], y_2)
y_pred_2 = pca_model.predict(X_pca[:,:-1])
kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
clusters_upenn = kmeans.fit_predict(X_pca[:,:-1])

fig, ax = plt.subplots(1, 4, figsize=(16,5), constrained_layout=True)

sc0 = ax[0].scatter(X_pca[:, 0], X_pca[:, 1], c=DATA.loc[DATA["site"] == cohort, "volume"], cmap="viridis", marker='o', alpha=0.7)
ax[0].spines[["top", "right"]].set_visible(False)
ax[0].set_xlim([-150,250])
ax[0].set_ylim([-150,250])
ax[0].set_xlabel(f"PC1 ({explained_variance[0]*100:.1f}%)")
ax[0].set_ylabel(f"PC2 ({explained_variance[1]*100:.1f}%)")

sc1 = ax[1].scatter(X_pca[:, 0], X_pca[:, 1], c=DATA.loc[DATA["site"] == cohort, "tdi"], cmap="viridis", marker='o', alpha=0.7)
ax[1].spines[["top", "right", "left"]].set_visible(False)
ax[1].set_xlim([-150,250])
ax[1].set_ylim([-150,250])
ax[1].set_yticks([])
ax[1].set_xlabel(f"PC1 ({explained_variance[0]*100:.1f}%)")

# Plot isocontours (lines of constant L-TDI)
norm = mcolors.Normalize(vmin=y_1.min(), vmax=y_1.max())  
cmap = cm.viridis  
ltdi_values = [2, 4, 6, 8]
x_ranges = [np.array([-100, -10]), np.array([-90, 70]), np.array([-40, 150]), np.array([60, 220])]
for iso, x_range in zip(ltdi_values, x_ranges):
    color = cmap(norm(iso))  
    y_range = (iso - pca_model.intercept_ - pca_model.coef_[0] * x_range) / pca_model.coef_[1]
    ax[2].plot(x_range, y_range, linewidth=8, color='black', alpha=1, zorder=1)  
    ax[2].plot(x_range, y_range, linewidth=4, color=color, alpha=0.75, zorder=2)
sc2 = ax[2].scatter(X_pca[:, 0], X_pca[:, 1], c=DATA.loc[DATA["site"] == cohort, "ltdi"], cmap="viridis", marker='o', alpha=0.5)
arrow_length = 200
arrow_dx = arrow_length 
arrow_dy = arrow_length *pca_model.coef_[1] / pca_model.coef_[0] 
ax[2].arrow(
    -0, 
    -0, 
    arrow_dx, 
    arrow_dy, 
    color="black", 
    head_width=5, 
    head_length=5, 
    linewidth=2.5, 
    alpha=0.75, 
    label=f"Rotation of {round(np.rad2deg(np.arctan(pca_model.coef_[1] / pca_model.coef_[0] )),2)}º"
)
ax[2].spines[["top", "right"]].set_visible(False)
ax[2].set_xlim([-150,250])
ax[2].set_ylim([-150,250])
ax[2].set_xlabel(f"PC1 ({explained_variance[0]*100:.1f}%)")
ax[2].set_ylabel(f"PC2 ({explained_variance[1]*100:.1f}%)")
ax[2].legend(frameon=False, loc="upper right")

sc3 = ax[3].scatter(y_pred_2, y_2, marker='o', alpha=0.7, c='tab:purple', s=10)
ax[3].plot([0, max(y_pred_2)], [0, max(y_pred_2)], 'k-', lw=2, alpha=0.5)
ax[3].text(0.10, 0.90, r'$R^2$'+f" = {round(pca_model.score(X_pca[:,:-1],y_2),2)}", transform=ax[3].transAxes, 
                    fontsize=10, verticalalignment='top', bbox=dict(boxstyle="round", alpha=0.1), color="black")   
ax[3].spines[["top", "right"]].set_visible(False)
ax[3].set_xlim([-1,10])
ax[3].set_ylim([-1,14])
ax[3].set_xlabel(f"L-TDI ~ {round(pca_model.coef_[0],3)}·PC1 + {round(pca_model.coef_[1],3)}·PC2 + {round(pca_model.intercept_,3)}")
ax[3].set_ylabel("L-TDI")

# Add colorbars below each subplot
cbar0 = fig.colorbar(sc0, ax=ax[0], orientation="horizontal", fraction=0.05, pad=0.02)
cbar0.set_label("Tumor volume (" + r'$cm^3$' + ")")

cbar1 = fig.colorbar(sc1, ax=ax[1], orientation="horizontal", fraction=0.05, pad=0.02)
cbar1.set_label("TDI (a.u.)")

cbar2 = fig.colorbar(sc2, ax=ax[2], orientation="horizontal", fraction=0.05, pad=0.02)
cbar2.set_label("L-TDI (a.u.)")

fig.suptitle(f"UPENN Cohort (N={all_upenn})", fontweight="bold")
fig.savefig(os.path.join(figs_folder_UPENN, f"Scatter-distributions_LTDI-TDI-Volume_UPENN-PCA.{fmt}"), dpi=dpi, format=fmt)

#### Joint cohorts

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 4))

# Plot for UCSF
sc = ax.scatter(
    DATA["tdi"], 
    DATA["ltdi"], 
    c=DATA["volume"], 
    cmap="viridis", 
    marker='o', 
    alpha=.75, 
    s=55
)
ax.spines[["top","right"]].set_visible(False)
ax.set_xlim([-5,320])
ax.set_ylim([-.1,12])
ax.spines['bottom'].set_bounds(0, 300)
ax.spines['left'].set_bounds(0, 12)
ax.set_xlabel("Tract density index (TDI)", fontsize=12)
ax.set_ylabel("Lesion-tract density index (L-TDI)", fontsize=12)
ax.set_title(f"Joint Cohorts (N={all_upenn+all_ucsf})", fontweight="bold")

# Create a separate axis for the colorbar
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
cbar = fig.colorbar(sc, cax=cax)
cbar.set_label("Tumor volume ("+r'$cm^3$'+")", fontsize=12)

fig.tight_layout()
fig.savefig(os.path.join(figs_folder_Joint, f"Scatter-distributions_LTDI-TDI-Volume_Joint.{fmt}"), dpi=dpi, format=fmt)

In [ ]:
X = DATA[["tdi", "ltdi", "volume"]].dropna().values  

# Apply PCA
pca = PCA(n_components=3)  
X_pca = pca.fit_transform(X)
explained_variance = pca.explained_variance_ratio_
y_2 = DATA["ltdi"]
pca_model = LinearRegression()
pca_model.fit(X_pca[:,:-1], y_2)
y_pred_2 = pca_model.predict(X_pca[:,:-1])

fig, ax = plt.subplots(1, 4, figsize=(16,5), constrained_layout=True)

sc0 = ax[0].scatter(X_pca[:, 0], X_pca[:, 1], c=DATA["volume"], cmap="viridis", marker='o', alpha=0.75)
ax[0].spines[["top", "right"]].set_visible(False)
ax[0].set_xlim([-150,250])
ax[0].set_ylim([-150,250])
ax[0].set_xlabel(f"PC1 ({explained_variance[0]*100:.1f}%)")
ax[0].set_ylabel(f"PC2 ({explained_variance[1]*100:.1f}%)")

sc1 = ax[1].scatter(X_pca[:, 0], X_pca[:, 1], c=DATA["tdi"], cmap="viridis", marker='o', alpha=0.75)
ax[1].spines[["top", "right", "left"]].set_visible(False)
ax[1].set_xlim([-150,250])
ax[1].set_ylim([-150,250])
ax[1].set_yticks([])
ax[1].set_xlabel(f"PC1 ({explained_variance[0]*100:.1f}%)")

# Plot isocontours (lines of constant L-TDI)
norm = mcolors.Normalize(vmin=y_1.min(), vmax=y_1.max())  
cmap = cm.viridis  
ltdi_values = [2, 4, 6, 8]
x_ranges = [np.array([-110, -10]), np.array([-90, 70]), np.array([-40, 150]), np.array([60, 220])]
for iso, x_range in zip(ltdi_values, x_ranges):
    color = cmap(norm(iso))  
    y_range = (iso - pca_model.intercept_ - pca_model.coef_[0] * x_range) / pca_model.coef_[1]
    ax[2].plot(x_range, y_range, linewidth=8, color='black', alpha=1, zorder=1)  
    ax[2].plot(x_range, y_range, linewidth=4, color=color, alpha=0.75, zorder=2)
sc2 = ax[2].scatter(X_pca[:, 0], X_pca[:, 1], c=DATA["ltdi"], cmap="viridis", marker='o', alpha=0.75)
arrow_length = 200
arrow_dx = arrow_length 
arrow_dy = arrow_length *pca_model.coef_[1] / pca_model.coef_[0] 
ax[2].arrow(
    -0, 
    -0, 
    arrow_dx, 
    arrow_dy, 
    color="black", 
    head_width=5, 
    head_length=5, 
    linewidth=2.5, 
    alpha=0.75, 
    label=f"Rotation of {round(np.rad2deg(np.arctan(pca_model.coef_[1] / pca_model.coef_[0] )),2)}º"
)
ax[2].spines[["top", "right"]].set_visible(False)
ax[2].set_xlim([-150,250])
ax[2].set_ylim([-150,250])
ax[2].set_xlabel(f"PC1 ({explained_variance[0]*100:.1f}%)")
ax[2].set_ylabel(f"PC2 ({explained_variance[1]*100:.1f}%)")
ax[2].legend(frameon=False, loc="upper right")

sc3 = ax[3].scatter(y_pred_2, y_2, marker='o', alpha=0.7, c='gray', s=10)
ax[3].plot([0, max(y_pred_2)], [0, max(y_pred_2)], 'k-', lw=2, alpha=0.5)
ax[3].text(0.10, 0.90, r'$R^2$'+f" = {round(pca_model.score(X_pca[:,:-1],y_2),2)}", transform=ax[3].transAxes, 
                    fontsize=10, verticalalignment='top', bbox=dict(boxstyle="round", alpha=0.1), color="black")   
ax[3].spines[["top", "right"]].set_visible(False)
ax[3].set_xlim([-1,10])
ax[3].set_ylim([-1,14])
ax[3].set_xlabel(f"L-TDI ~ {round(pca_model.coef_[0],3)}·PC1 + {round(pca_model.coef_[1],3)}·PC2 + {round(pca_model.intercept_,3)}")
ax[3].set_ylabel("L-TDI")

# Add colorbars below each subplot
cbar0 = fig.colorbar(sc0, ax=ax[0], orientation="horizontal", fraction=0.05, pad=0.02)
cbar0.set_label("Tumor volume (" + r'$cm^3$' + ")")

cbar1 = fig.colorbar(sc1, ax=ax[1], orientation="horizontal", fraction=0.05, pad=0.02)
cbar1.set_label("TDI (a.u.)")

cbar2 = fig.colorbar(sc2, ax=ax[2], orientation="horizontal", fraction=0.05, pad=0.02)
cbar2.set_label("L-TDI (a.u.)")

fig.suptitle(f"Joined Cohorts (N={all_upenn+all_ucsf})", fontweight="bold")
fig.savefig(os.path.join(figs_folder_Joint, f"Scatter-distributions_LTDI-TDI-Volume_Joint-PCA.{fmt}"), dpi=dpi, format=fmt)

### Anatomical underpinnings of tract density-based markers

#### UCSF Cohort

In [ ]:
os.makedirs(os.path.join(figs_folder_UCSF,"NIFTIs"), exist_ok=True)

In [ ]:
# Execitopm time around 0.1 mins if the results can be loaded
# Execution time around 7-8 mins if the data needs to be computed from scratch
tumor_masks = glob.glob(f"{root_ucsf}/TDMaps_Grade-IV/*/masks/*-whole*")
cohort = 1

small_small = ucsf["ID"].values[KMEANS_cohorts[cohort].labels_==2]
small_big = ucsf["ID"].values[KMEANS_cohorts[cohort].labels_==0]
big_medium = ucsf["ID"].values[KMEANS_cohorts[cohort].labels_==1]
def rewrite_subjectID(subject_ID):
    subject_4digits = subject_ID.split("-")
    subject = "-".join(subject_4digits[:-1])
    digits = str(subject_4digits[-1][1:])
    return subject + "-" + digits

try:
    print("Loading pre-existing results ...")
    uc_masks_ss = nib.load(os.path.join(figs_folder_UCSF, "NIFTIs", "Lesions__Cluster-2_Vlm-small_TDI-small.nii.gz")).get_fdata()
    uc_masks_sb = nib.load(os.path.join(figs_folder_UCSF, "NIFTIs", "Lesions__Cluster-0_Vlm-small_TDI-big.nii.gz")).get_fdata()
    uc_masks_bm = nib.load(os.path.join(figs_folder_UCSF, "NIFTIs", "Lesions__Cluster-1_Vlm-big_TDI-medium.nii.gz")).get_fdata()
    print("Done!")
except:
    print("Loading data failed! Creating the samples from the data")
    Nss, Nsb, Nbm = 0, 0, 0
    for file in tumor_masks:
        id = file.split("_")[-2][-14:]
        if rewrite_subjectID(id) in small_small:
            if Nss == 0:
                uc_masks_ss = nib.load(file).get_fdata()
                Nss += 1
            else:
                uc_masks_ss += nib.load(file).get_fdata()
                Nss += 1
        
        if rewrite_subjectID(id) in small_big:
            if Nsb == 0:
                uc_masks_sb = nib.load(file).get_fdata()
                Nsb += 1
            else:
                uc_masks_sb += nib.load(file).get_fdata()
                Nsb += 1
            
            
        if rewrite_subjectID(id) in big_medium:
            if Nbm == 0:
                uc_masks_bm = nib.load(file).get_fdata()
                Nbm += 1
            else:
                uc_masks_bm += nib.load(file).get_fdata()
                Nbm += 1
        
    print("--------------------------------------")
    print("Saving files ...")
    nib.save(nib.Nifti1Image(uc_masks_ss, T1.affine, T1.header), os.path.join(figs_folder_UCSF, "NIFTIs", "Lesions__Cluster-2_Vlm-small_TDI-small.nii.gz"))
    nib.save(nib.Nifti1Image(uc_masks_sb, T1.affine, T1.header), os.path.join(figs_folder_UCSF, "NIFTIs", "Lesions__Cluster-0_Vlm-small_TDI-big.nii.gz"))
    nib.save(nib.Nifti1Image(uc_masks_bm, T1.affine, T1.header), os.path.join(figs_folder_UCSF, "NIFTIs", "Lesions__Cluster-1_Vlm-big_TDI-medium.nii.gz"))

    print(f"Number of samples in clsuter 0: {Nbm}")
    print(f"Number of samples in clsuter 1: {Nsb}")
    print(f"Number of samples in clsuter 2: {Nss}")
    print("--------------------------------------")
    print(f"Total number of samples: {Nss+Nsb+Nbm}")

In [ ]:
# Execitopm time around 10-12 mins if the results can be loaded
# Execution time around 20-25 mins if the data needs to be computed from scratch
ltdis = glob.glob(f"{root_ucsf}/TDMaps_Grade-IV/*/maps/*-whole_TDMap-lesion*")

# We reiterate the definition of the groups just to be sure
small_small = ucsf["ID"].values[KMEANS_cohorts[1].labels_==2]
small_big = ucsf["ID"].values[KMEANS_cohorts[1].labels_==0]
big_medium = ucsf["ID"].values[KMEANS_cohorts[1].labels_==1]
def rewrite_subjectID(subject_ID):
    subject_4digits = subject_ID.split("-")
    subject = "-".join(subject_4digits[:-1])
    digits = str(subject_4digits[-1][1:])
    return subject + "-" + digits

try:
    print("Loading pre-existing results ...")
    uc_Ltracts_ss = nib.load(os.path.join(figs_folder_UCSF, "NIFTIs", "Ltracts__Cluster-2_Vlm-small_TDI-small.nii.gz")).get_fdata()
    uc_Ltracts_sb = nib.load(os.path.join(figs_folder_UCSF, "NIFTIs", "Ltracts__Cluster-0_Vlm-small_TDI-big.nii.gz")).get_fdata()
    uc_Ltracts_bm = nib.load(os.path.join(figs_folder_UCSF, "NIFTIs", "Ltracts__Cluster-1_Vlm-big_TDI-medium.nii.gz")).get_fdata()
    Nss, Nsb, Nbm = len(uc_Ltracts_ss), len(uc_Ltracts_sb), len(uc_Ltracts_bm)

    uc_Ltracts_ss_mean = nib.load(os.path.join(figs_folder_UCSF, "NIFTIs", "Ltracts-mean__Cluster-2_Vlm-small_TDI-small.nii.gz")).get_fdata()
    uc_Ltracts_sb_mean = nib.load(os.path.join(figs_folder_UCSF, "NIFTIs", "Ltracts-mean__Cluster-0_Vlm-small_TDI-big.nii.gz")).get_fdata()
    uc_Ltracts_bm_mean = nib.load(os.path.join(figs_folder_UCSF, "NIFTIs", "Ltracts-mean__Cluster-1_Vlm-big_TDI-medium.nii.gz")).get_fdata()

    uc_Ltracts_ss_std = nib.load(os.path.join(figs_folder_UCSF, "NIFTIs", "Ltracts-std__Cluster-2_Vlm-small_TDI-small.nii.gz")).get_fdata()
    uc_Ltracts_sb_std = nib.load(os.path.join(figs_folder_UCSF, "NIFTIs", "Ltracts-std__Cluster-0_Vlm-small_TDI-big.nii.gz")).get_fdata()
    uc_Ltracts_bm_std = nib.load(os.path.join(figs_folder_UCSF, "NIFTIs", "Ltracts-std__Cluster-1_Vlm-big_TDI-medium.nii.gz")).get_fdata()
except:
    print("Loading data failed! Creating the samples from the data")
    Nss, Nsb, Nbm = 0, 0, 0
    uc_Ltracts_ss, uc_Ltracts_sb, uc_Ltracts_bm = [], [], []
    for file in tqdm(ltdis):
        id = file.split("_")[-3][-14:]
        id = rewrite_subjectID(id)

        ldata = nib.load(file).get_fdata()
        if id in small_small:
            uc_Ltracts_ss.append(ldata)
            Nss += 1
        
        if id in small_big:
            uc_Ltracts_sb.append(ldata)
            Nsb += 1        
            
        if id in big_medium:
            uc_Ltracts_bm.append(ldata)
            Nbm += 1

    print("--------------------------------------")
    print("Computing the mean and std tract density per voxel ...")
    uc_Ltracts_ss, uc_Ltracts_sb, uc_Ltracts_bm = np.array(uc_Ltracts_ss), np.array(uc_Ltracts_sb), np.array(uc_Ltracts_bm)
    uc_Ltracts_ss_mean, uc_Ltracts_sb_mean, uc_Ltracts_bm_mean = uc_Ltracts_ss.mean(axis=0), uc_Ltracts_sb.mean(axis=0), uc_Ltracts_bm.mean(axis=0)
    uc_Ltracts_ss_std, uc_Ltracts_sb_std, uc_Ltracts_bm_std = uc_Ltracts_ss.std(axis=0), uc_Ltracts_sb.std(axis=0), uc_Ltracts_bm.std(axis=0)

    print("--------------------------------------")
    print("Saving files ...")
    nib.save(nib.Nifti1Image(uc_Ltracts_ss, T1.affine, T1.header), os.path.join(figs_folder_UCSF, "NIFTIs", "Ltracts__Cluster-2_Vlm-small_TDI-small.nii.gz"))
    nib.save(nib.Nifti1Image(uc_Ltracts_ss_mean, T1.affine, T1.header), os.path.join(figs_folder_UCSF, "NIFTIs" "Ltracts-mean__Cluster-2_Vlm-small_TDI-small.nii.gz"))
    nib.save(nib.Nifti1Image(uc_Ltracts_ss_std, T1.affine, T1.header), os.path.join(figs_folder_UCSF, "NIFTIs", "Ltracts-std__Cluster-2_Vlm-small_TDI-small.nii.gz"))

    nib.save(nib.Nifti1Image(uc_Ltracts_sb, T1.affine, T1.header), os.path.join(figs_folder_UCSF, "NIFTIs", "Ltracts__Cluster-0_Vlm-small_TDI-big.nii.gz"))
    nib.save(nib.Nifti1Image(uc_Ltracts_sb_mean, T1.affine, T1.header), os.path.join(figs_folder_UCSF, "NIFTIs", "Ltracts-mean__Cluster-0_Vlm-small_TDI-big.nii.gz"))
    nib.save(nib.Nifti1Image(uc_Ltracts_sb_std, T1.affine, T1.header), os.path.join(figs_folder_UCSF, "NIFTIs", "Ltracts-std__Cluster-0_Vlm-small_TDI-big.nii.gz"))

    nib.save(nib.Nifti1Image(uc_Ltracts_bm, T1.affine, T1.header), os.path.join(figs_folder_UCSF, "NIFTIs", "Ltracts__Cluster-1_Vlm-big_TDI-medium.nii.gz"))
    nib.save(nib.Nifti1Image(uc_Ltracts_bm_mean, T1.affine, T1.header), os.path.join(figs_folder_UCSF, "NIFTIs", "Ltracts-mean__Cluster-1_Vlm-big_TDI-medium.nii.gz"))
    nib.save(nib.Nifti1Image(uc_Ltracts_bm_std, T1.affine, T1.header), os.path.join(figs_folder_UCSF, "NIFTIs", "Ltracts-std__Cluster-1_Vlm-big_TDI-medium.nii.gz"))

print("--------------------------------------")
print(f"Number of samples in clsuter 0 (big-medium): {Nbm}")
print(f"Number of samples in clsuter 1 (small-big): {Nsb}")
print(f"Number of samples in clsuter 2 (small-small): {Nss}")
print("======================================")
print(f"Total number of samples: {Nss+Nsb+Nbm}")

In [ ]:
# The NIFTI images are saved (and can be loaded instead of computed everytime). These two thresholds are used to plot the maps in pdf format for rapid inspection
mask_threshold = 5
tract_threshold = 15
std_threshold = 10

plot_lesions(uc_masks_ss, T1, save_figname=os.path.join(figs_folder_UCSF, "Lesions__Cluster-2_Vlm-small_TDI-small.pdf"), title=None, color_map="Purples", threshold=mask_threshold, Nslices=15, dpi=400)
plot_lesions(uc_Ltracts_ss_mean, T1, save_figname=os.path.join(figs_folder_UCSF, "Ltracts-mean__Cluster-2_Vlm-small_TDI-small.pdf"), title=None, color_map="Purples", threshold=tract_threshold, Nslices=15, dpi=400)
plot_lesions(uc_Ltracts_ss_std, T1, save_figname=os.path.join(figs_folder_UCSF, "Ltracts-std__Cluster-2_Vlm-small_TDI-small.pdf"), title=None, color_map="viridis", threshold=std_threshold, Nslices=15, dpi=400)

In [ ]:
# The NIFTI images are saved (and can be loaded instead of computed everytime). These two thresholds are used to plot the maps in pdf format for rapid inspection
mask_threshold = 15
tract_threshold = 15
std_threshold = 10

plot_lesions(uc_masks_sb, T1, save_figname=os.path.join(figs_folder_UCSF, "Lesions__Cluster-0_Vlm-small_TDI-big.pdf"), title=None, color_map="Oranges", threshold=mask_threshold, Nslices=15, dpi=400)
plot_lesions(uc_Ltracts_sb_mean, T1, save_figname=os.path.join(figs_folder_UCSF, "Ltracts-mean__Cluster-0_Vlm-small_TDI-big.pdf"), title=None, color_map="Oranges", threshold=tract_threshold, Nslices=15, dpi=400)
plot_lesions(uc_Ltracts_sb_std, T1, save_figname=os.path.join(figs_folder_UCSF, "Ltracts-std__Cluster-0_Vlm-small_TDI-big.pdf"), title=None, color_map="viridis", threshold=std_threshold, Nslices=15, dpi=400)

In [ ]:
# The NIFTI images are saved (and can be loaded instead of computed everytime). These two thresholds are used to plot the maps in pdf format for rapid inspection
mask_threshold = 15
tract_threshold = 15
std_threshold = 10

plot_lesions(uc_masks_bm, T1, save_figname=os.path.join(figs_folder_UCSF, "Lesions__Cluster-1_Vlm-big_TDI-medium.pdf"), title=None, color_map="BuGn", threshold=mask_threshold, Nslices=15, dpi=400)
plot_lesions(uc_Ltracts_bm_mean, T1, save_figname=os.path.join(figs_folder_UCSF, "Ltracts-mean__Cluster-1_Vlm-big_TDI-medium.pdf"), title=None, color_map="BuGn", threshold=tract_threshold, Nslices=15, dpi=400)
plot_lesions(uc_Ltracts_bm_std, T1, save_figname=os.path.join(figs_folder_UCSF, "Ltracts-std__Cluster-1_Vlm-big_TDI-medium.pdf"), title=None, color_map="viridis", threshold=std_threshold, Nslices=15, dpi=400)

In [ ]:
# The NIFTI images are saved (and can be loaded instead of computed everytime). These two thresholds are used to plot the maps in pdf format for rapid inspection
ov_tract_threshold = 20
bin_threshold = 0

plot_lesions(
    np.where(uc_Ltracts_ss>bin_threshold, 1, 0).sum(axis=0), 
    T1, 
    save_figname=os.path.join(figs_folder_UCSF, "Ltracts__Cluster-2_Vlm-small_TDI-small.pdf"), 
    title=None, 
    color_map="Purples", 
    threshold=ov_tract_threshold, 
    Nslices=15, 
    dpi=400
)
plot_lesions(
    np.where(uc_Ltracts_sb>bin_threshold, 1, 0).sum(axis=0), 
    T1, 
    save_figname=os.path.join(figs_folder_UCSF, "Ltracts__Cluster-0_Vlm-small_TDI-big.pdf"), 
    title=None, 
    color_map="Oranges", 
    threshold=ov_tract_threshold, 
    Nslices=15, 
    dpi=400
)
plot_lesions(
    np.where(uc_Ltracts_bm>bin_threshold, 1, 0).sum(axis=0), 
    T1, 
    save_figname=os.path.join(figs_folder_UCSF, "Ltracts__Cluster-1_Vlm-big_TDI-medium.pdf"), 
    title=None, 
    color_map="BuGn", 
    threshold=ov_tract_threshold, 
    Nslices=15, 
    dpi=400
)

In [ ]:
months = range(0,80,10)
colors = ['#d95f02', '#1b9e77', '#7570b3'] # Orange, Teal, Purple
labels = ['Small volume & Large TDI', 'Large volume & Medium TDI', 'Small volume & Small TDI']
cohort = 1

fig, ax = plt.subplots(1,1, figsize=(6,6))
STATS = []
G_STATS = []
nums = np.empty(shape=(3,), dtype=object)
for i, D in enumerate(np.unique(KMEANS_cohorts[cohort].labels_)):
    cohort_data = DATA[DATA["site"] == cohort]
    diag = cohort_data[KMEANS_cohorts[cohort].labels_==D]
    mask = ~np.isnan(diag["status"]) & ~np.isnan(diag["OS"])
    diag = diag[mask]
    nums[i] = []
    for t in months:
        nums[i].append((diag["OS"]>=(t*daysXmonth)).sum())
    
    time, survival_prob, conf_int = kaplan_meier_estimator(
                diag["status"]==1, diag["OS"], conf_type="log-log"
            )
    ax.step(time/daysXmonth, survival_prob, where="post", label=f"{labels[i]} (Cluster {D})", color=colors[i], alpha=0.75)
    ax.fill_between(time/daysXmonth, conf_int[0], conf_int[1], alpha=0.15, step="post", color=colors[i])
    for t in diag.loc[diag["status"]==0, "OS"].values: # Censoring times
        ax.plot(time[time==t]/daysXmonth, survival_prob[time==t], "|", color=colors[i])

    STATS.extend([(st, ovs) for st, ovs in zip(diag["status"]==1, diag["OS"].values)])
    G_STATS.extend([i+1 for ovs in diag["OS"].values])

STATS = np.array(STATS, dtype=[('event', 'bool'),('time', 'float')])
chisquared, p_val, stats, covariance = compare_survival(STATS, G_STATS, return_stats=True)
ax.text(0.70, 0.60, r"$\chi^2 =$"+f"{round(chisquared,4)}, \np = {round(p_val,4)}", transform=ax.transAxes, 
                    fontsize=10, verticalalignment='top', bbox=dict(boxstyle="round", alpha=0.1), color="red" if p_val<=0.05 else "black")    

for i,t in enumerate(months):
    ax.text(t-2, -0.07, f"{nums[0][i]}", transform=ax.transData, fontsize=11, verticalalignment='top', color=colors[0]) 
    ax.text(t-2, -0.13, f"{nums[1][i]}", transform=ax.transData, fontsize=11, verticalalignment='top', color=colors[1]) 
    ax.text(t-2, -0.19, f"{nums[2][i]}", transform=ax.transData, fontsize=11, verticalalignment='top', color=colors[2]) 
ax.text(-2, -0.01, "No. at risk", transform=ax.transData, fontsize=11, verticalalignment='top', color="black", fontweight='bold') 
ax.hlines(0,-5, months[-1], color="black", linewidth=.5)
ax.set_ylim([-.26,1])
ax.set_xlim([-5,months[-1]])
ax.set_xticks(range(0,months[-1]+10,10))
ax.set_xticklabels(range(0,months[-1]+10,10))
ax.set_yticks([0,0.2,0.4,0.6,0.8,1])
ax.set_yticklabels([0,0.2,0.4,0.6,0.8,1])
ax.spines['left'].set_bounds(0,1)
ax.spines['bottom'].set_bounds(0,months[-1])
ax.set_xlabel("Time (months)", fontsize=12)
ax.set_ylabel("Overall survival", fontsize=12)
ax.spines[["top", "right"]].set_visible(False)
ax.legend(loc="upper right")

fig.tight_layout()
fig.savefig(os.path.join(figs_folder_UCSF, f"Clustered-groups_Survival-times.{fmt}"), dpi=dpi, format=fmt)

G_STATS_array = np.array(G_STATS)
vals = []
for group in [1,2,3]:
    chisquared, p_val, stats, covariance = compare_survival(STATS[G_STATS_array!=group], G_STATS_array[G_STATS_array!=group], return_stats=True)
    print(f"Ommited cluster {group-1}: Statistic = {chisquared} and p-value = {p_val}")
    vals.append(p_val)

_, corrected = fdrcorrection(vals, alpha=0.05, method='p', is_sorted=False)
print("FDR corrected results: ", corrected)

#### UPENN Cohort

In [ ]:
os.makedirs(os.path.join(figs_folder_UPENN,"NIFTIs"), exist_ok=True)

In [ ]:
# Execitopm time around 0.1 mins if the results can be loaded
# Execution time around 10 mins if the data needs to be computed from scratch
tumor_masks = glob.glob(f"{root_upenn}/TDMaps_IDH1-WT/*/masks/*-whole*")
cohort = 2

small_small = upenn["ID"].values[KMEANS_cohorts[cohort].labels_==0]
small_big = upenn["ID"].values[KMEANS_cohorts[cohort].labels_==2]
big_medium = upenn["ID"].values[KMEANS_cohorts[cohort].labels_==1]

try:
    print("Loading pre-existing results ...")
    up_masks_ss = nib.load(os.path.join(figs_folder_UPENN, "NIFTIs", "Lesions__Cluster-0_Vlm-small_TDI-small.nii.gz")).get_fdata()
    up_masks_sb = nib.load(os.path.join(figs_folder_UPENN, "NIFTIs", "Lesions__Cluster-2_Vlm-small_TDI-big.nii.gz")).get_fdata()
    up_masks_bm = nib.load(os.path.join(figs_folder_UPENN, "NIFTIs", "Lesions__Cluster-1_Vlm-big_TDI-medium.nii.gz")).get_fdata()
    print("Done!")
except:
    print("Loading data failed! Creating the samples from the data")    
    Nss, Nsb, Nbm = 0, 0, 0
    for file in tumor_masks:
        id = file.split("/")[-1].split(".")[0].split("tissue")[0][-19:-1]
        if id in small_small:
            if Nss == 0:
                up_masks_ss = nib.load(file).get_fdata()
                Nss += 1
            else:
                up_masks_ss += nib.load(file).get_fdata()
                Nss += 1
        
        if id in small_big:
            if Nsb == 0:
                up_masks_sb = nib.load(file).get_fdata()
                Nsb += 1
            else:
                up_masks_sb += nib.load(file).get_fdata()
                Nsb += 1
            
            
        if id in big_medium:
            if Nbm == 0:
                up_masks_bm = nib.load(file).get_fdata()
                Nbm += 1
            else:
                up_masks_bm += nib.load(file).get_fdata()
                Nbm += 1

    print("--------------------------------------")
    print("Saving files ...")
    nib.save(nib.Nifti1Image(up_masks_ss, T1.affine, T1.header), os.path.join(figs_folder_UPENN, "NIFTIs", "Lesions__Cluster-0_Vlm-small_TDI-small.nii.gz"))
    nib.save(nib.Nifti1Image(up_masks_sb, T1.affine, T1.header), os.path.join(figs_folder_UPENN, "NIFTIs", "Lesions__Cluster-2_Vlm-small_TDI-big.nii.gz"))
    nib.save(nib.Nifti1Image(up_masks_bm, T1.affine, T1.header), os.path.join(figs_folder_UPENN, "NIFTIs", "Lesions__Cluster-1_Vlm-big_TDI-medium.nii.gz"))

    print(f"Number of samples in clsuter 0: {Nbm}")
    print(f"Number of samples in clsuter 1: {Nsb}")
    print(f"Number of samples in clsuter 2: {Nss}")
    print("--------------------------------------")
    print(f"Total number of samples: {Nss+Nsb+Nbm}")

In [ ]:
# Execitopm time around 10-15 mins if the results can be loaded
# Execution time around 20-25 mins if the data needs to be computed from scratch
ltdis = glob.glob(f"{root_upenn}/TDMaps_IDH1-WT/*/maps/*-whole_TDMap-lesion*")
cohort = 2

# We reiterate the definition of the groups just to be sure
small_small = upenn["ID"].values[KMEANS_cohorts[cohort].labels_==0]
small_big = upenn["ID"].values[KMEANS_cohorts[cohort].labels_==2]
big_medium = upenn["ID"].values[KMEANS_cohorts[cohort].labels_==1]

try:
    print("Loading pre-existing results ...")
    up_Ltracts_ss = nib.load(os.path.join(figs_folder_UPENN, "NIFTIs", "Ltracts__Cluster-0_Vlm-small_TDI-small.nii.gz")).get_fdata()
    up_Ltracts_sb = nib.load(os.path.join(figs_folder_UPENN, "NIFTIs", "Ltracts__Cluster-2_Vlm-small_TDI-big.nii.gz")).get_fdata()
    up_Ltracts_bm = nib.load(os.path.join(figs_folder_UPENN, "NIFTIs", "Ltracts__Cluster-1_Vlm-big_TDI-medium.nii.gz")).get_fdata()
    Nss, Nsb, Nbm = len(up_Ltracts_ss), len(up_Ltracts_sb), len(up_Ltracts_bm)

    up_Ltracts_ss_mean = nib.load(os.path.join(figs_folder_UPENN, "NIFTIs", "Ltracts-mean__Cluster-0_Vlm-small_TDI-small.nii.gz")).get_fdata()
    up_Ltracts_sb_mean = nib.load(os.path.join(figs_folder_UPENN, "NIFTIs", "Ltracts-mean__Cluster-2_Vlm-small_TDI-big.nii.gz")).get_fdata()
    up_Ltracts_bm_mean = nib.load(os.path.join(figs_folder_UPENN, "NIFTIs", "Ltracts-mean__Cluster-1_Vlm-big_TDI-medium.nii.gz")).get_fdata()

    up_Ltracts_ss_std = nib.load(os.path.join(figs_folder_UPENN, "NIFTIs", "Ltracts-std__Cluster-0_Vlm-small_TDI-small.nii.gz")).get_fdata()
    up_Ltracts_sb_std = nib.load(os.path.join(figs_folder_UPENN, "NIFTIs", "Ltracts-std__Cluster-2_Vlm-small_TDI-big.nii.gz")).get_fdata()
    up_Ltracts_bm_std = nib.load(os.path.join(figs_folder_UPENN, "NIFTIs", "Ltracts-std__Cluster-1_Vlm-big_TDI-medium.nii.gz")).get_fdata()
except:
    print("Loading data failed! Creating the samples from the data")
    Nss, Nsb, Nbm = 0, 0, 0
    up_Ltracts_ss, up_Ltracts_sb, up_Ltracts_bm = [], [], []
    for file in tqdm(ltdis):
        id = file.split("/")[-1].split(".")[0].split("tissue")[0][-19:-1]
        
        ldata = nib.load(file).get_fdata()
        if id in small_small:
            up_Ltracts_ss.append(ldata)
            Nss += 1
        
        if id in small_big:
            up_Ltracts_sb.append(ldata)
            Nsb += 1        
            
        if id in big_medium:
            up_Ltracts_bm.append(ldata)
            Nbm += 1

    print("--------------------------------------")
    print("Computing the mean and std tract density per voxel ...")
    up_Ltracts_ss, up_Ltracts_sb, up_Ltracts_bm = np.array(up_Ltracts_ss), np.array(up_Ltracts_sb), np.array(up_Ltracts_bm)
    up_Ltracts_ss_mean, up_Ltracts_sb_mean, up_Ltracts_bm_mean = up_Ltracts_ss.mean(axis=0), up_Ltracts_sb.mean(axis=0), up_Ltracts_bm.mean(axis=0)
    up_Ltracts_ss_std, up_Ltracts_sb_std, up_Ltracts_bm_std = up_Ltracts_ss.std(axis=0), up_Ltracts_sb.std(axis=0), up_Ltracts_bm.std(axis=0)

    print("--------------------------------------")
    print("Saving files ...")
    nib.save(nib.Nifti1Image(up_Ltracts_ss, T1.affine, T1.header), os.path.join(figs_folder_UPENN, "NIFTIs", "Ltracts__Cluster-0_Vlm-small_TDI-small.nii.gz"))
    nib.save(nib.Nifti1Image(up_Ltracts_ss_mean, T1.affine, T1.header), os.path.join(figs_folder_UPENN, "NIFTIs", "Ltracts-mean__Cluster-0_Vlm-small_TDI-small.nii.gz"))
    nib.save(nib.Nifti1Image(up_Ltracts_ss_std, T1.affine, T1.header), os.path.join(figs_folder_UPENN, "NIFTIs", "Ltracts-std__Cluster-0_Vlm-small_TDI-small.nii.gz"))

    nib.save(nib.Nifti1Image(up_Ltracts_sb, T1.affine, T1.header), os.path.join(figs_folder_UPENN, "NIFTIs", "Ltracts__Cluster-2_Vlm-big_TDI-medium.nii.gz"))
    nib.save(nib.Nifti1Image(up_Ltracts_sb_mean, T1.affine, T1.header), os.path.join(figs_folder_UPENN, "NIFTIs", "Ltracts-mean__Cluster-2_Vlm-big_TDI-medium.nii.gz"))
    nib.save(nib.Nifti1Image(up_Ltracts_sb_std, T1.affine, T1.header), os.path.join(figs_folder_UPENN, "NIFTIs", "Ltracts-std__Cluster-2_Vlm-big_TDI-medium.nii.gz"))

    nib.save(nib.Nifti1Image(up_Ltracts_bm, T1.affine, T1.header), os.path.join(figs_folder_UPENN, "NIFTIs", "Ltracts__Cluster-1_Vlm-small_TDI-big.nii.gz"))
    nib.save(nib.Nifti1Image(up_Ltracts_bm_mean, T1.affine, T1.header), os.path.join(figs_folder_UPENN, "NIFTIs", "Ltracts-mean__Cluster-1_Vlm-small_TDI-big.nii.gz"))
    nib.save(nib.Nifti1Image(up_Ltracts_bm_std, T1.affine, T1.header), os.path.join(figs_folder_UPENN, "NIFTIs", "Ltracts-std__Cluster-1_Vlm-small_TDI-big.nii.gz"))

print("--------------------------------------")
print(f"Number of samples in clsuter 0 (big-medium): {Nbm}")
print(f"Number of samples in clsuter 1 (small-big): {Nsb}")
print(f"Number of samples in clsuter 2 (small-small): {Nss}")
print("======================================")
print(f"Total number of samples: {Nss+Nsb+Nbm}")

In [ ]:
# The NIFTI images are saved (and can be loaded instead of computed everytime). These two thresholds are used to plot the maps in pdf format for rapid inspection
mask_threshold = 5
tract_threshold = 15
std_threshold = 10

plot_lesions(up_masks_ss, T1, save_figname=os.path.join(figs_folder_UPENN, "Lesions__Cluster-0_Vlm-small_TDI-small.pdf"), title=None, color_map="Purples", threshold=mask_threshold, Nslices=15, dpi=400)
plot_lesions(up_Ltracts_ss_mean, T1, save_figname=os.path.join(figs_folder_UPENN, "Ltracts-mean__Cluster-0_Vlm-small_TDI-small.pdf"), title=None, color_map="Purples", threshold=tract_threshold, Nslices=15, dpi=400)
plot_lesions(up_Ltracts_ss_std, T1, save_figname=os.path.join(figs_folder_UPENN, "Ltracts-std__Cluster-0_Vlm-small_TDI-small.pdf"), title=None, color_map="viridis", threshold=std_threshold, Nslices=15, dpi=400)

In [ ]:
# The NIFTI images are saved (and can be loaded instead of computed everytime). These two thresholds are used to plot the maps in pdf format for rapid inspection
mask_threshold = 20
tract_threshold = 15
std_threshold = 10

plot_lesions(up_masks_sb, T1, save_figname=os.path.join(figs_folder_UPENN, "Lesions__Cluster-2_Vlm-small_TDI-big.pdf"), title=None, color_map="Oranges", threshold=mask_threshold, Nslices=15, dpi=400)
plot_lesions(up_Ltracts_sb_mean, T1, save_figname=os.path.join(figs_folder_UPENN, "Ltracts-mean__Cluster-2_Vlm-small_TDI-big.pdf"), title=None, color_map="Oranges", threshold=tract_threshold, Nslices=15, dpi=400)
plot_lesions(up_Ltracts_sb_std, T1, save_figname=os.path.join(figs_folder_UPENN, "Ltracts-std__Cluster-2_Vlm-small_TDI-big.pdf"), title=None, color_map="viridis", threshold=std_threshold, Nslices=15, dpi=400)

In [ ]:
# The NIFTI images are saved (and can be loaded instead of computed everytime). These two thresholds are used to plot the maps in pdf format for rapid inspection
mask_threshold = 20
tract_threshold = 15
std_threshold = 10

plot_lesions(up_masks_bm, T1, save_figname=os.path.join(figs_folder_UPENN, "Lesions__Cluster-1_Vlm-big_TDI-medium.pdf"), title=None, color_map="BuGn", threshold=mask_threshold, Nslices=15, dpi=400)
plot_lesions(up_Ltracts_bm_mean, T1, save_figname=os.path.join(figs_folder_UPENN, "Ltracts-mean__Cluster-1_Vlm-big_TDI-medium.pdf"), title=None, color_map="BuGn", threshold=tract_threshold, Nslices=15, dpi=400)
plot_lesions(up_Ltracts_bm_std, T1, save_figname=os.path.join(figs_folder_UPENN, "Ltracts-std__Cluster-1_Vlm-big_TDI-medium.pdf"), title=None, color_map="viridis", threshold=std_threshold, Nslices=15, dpi=400)

In [ ]:
# The NIFTI images are saved (and can be loaded instead of computed everytime). These two thresholds are used to plot the maps in pdf format for rapid inspection
ov_tract_threshold = 20
bin_threshold = 0

plot_lesions(
    np.where(up_Ltracts_ss>bin_threshold, 1, 0).sum(axis=0), 
    T1, 
    save_figname=os.path.join(figs_folder_UPENN, "Ltracts__Cluster-0_Vlm-small_TDI-small.pdf"), 
    title=None, 
    color_map="Purples", 
    threshold=ov_tract_threshold, 
    Nslices=15, 
    dpi=400
)
plot_lesions(
    np.where(up_Ltracts_sb>bin_threshold, 1, 0).sum(axis=0), 
    T1, 
    save_figname=os.path.join(figs_folder_UPENN, "Ltracts__Cluster-2_Vlm-small_TDI-big.pdf"), 
    title=None, 
    color_map="Oranges", 
    threshold=ov_tract_threshold, 
    Nslices=15, 
    dpi=400
)
plot_lesions(
    np.where(up_Ltracts_bm>bin_threshold, 1, 0).sum(axis=0), 
    T1, 
    save_figname=os.path.join(figs_folder_UPENN, "Ltracts__Cluster-1_Vlm-big_TDI-medium.pdf"), 
    title=None, 
    color_map="BuGn", 
    threshold=ov_tract_threshold, 
    Nslices=15, 
    dpi=400
)

In [ ]:
months = range(0,110,10)
colors = ['#7570b3', '#1b9e77', '#d95f02'] # Purple, Teal, Orange
labels = ['Small volume & Small TDI', 'Large volume & Medium TDI', 'Small volume & Large TDI']
cohort = 2

fig, ax = plt.subplots(1,1, figsize=(5,4))
STATS = []
G_STATS = []
nums = np.empty(shape=(3,), dtype=object)
for i, D in enumerate(np.unique(KMEANS_cohorts[cohort].labels_)):
    cohort_data = DATA[DATA["site"] == cohort]
    diag = cohort_data[KMEANS_cohorts[cohort].labels_==D]
    mask = ~np.isnan(diag["status"]) & ~np.isnan(diag["OS"])
    diag = diag[mask]
    nums[i] = []
    for t in months:
        nums[i].append((diag["OS"]>=(t*daysXmonth)).sum())
    
    time, survival_prob, conf_int = kaplan_meier_estimator(
                diag["status"]==1, diag["OS"], conf_type="log-log"
            )
    ax.step(time/daysXmonth, survival_prob, where="post", label=f"{labels[i]} (Cluster {D})", color=colors[i], alpha=0.75)
    ax.fill_between(time/daysXmonth, conf_int[0], conf_int[1], alpha=0.15, step="post", color=colors[i])
    for t in diag.loc[diag["status"]==0, "OS"].values: # Censoring times
        ax.plot(time[time==t]/daysXmonth, survival_prob[time==t], "|", color=colors[i])

    STATS.extend([(st, ovs) for st, ovs in zip(diag["status"]==1, diag["OS"].values)])
    G_STATS.extend([i+1 for ovs in diag["OS"].values])

STATS = np.array(STATS, dtype=[('event', 'bool'),('time', 'float')])
chisquared, p_val, stats, covariance = compare_survival(STATS, G_STATS, return_stats=True)
ax.text(0.70, 0.60, r"$\chi^2 =$"+f"{round(chisquared,4)}, \np = {round(p_val,4)}", transform=ax.transAxes, 
                    fontsize=10, verticalalignment='top', bbox=dict(boxstyle="round", alpha=0.1), color="red" if p_val<=0.05 else "black")    

for i,t in enumerate(months):
    ax.text(t-2, -0.07, f"{nums[0][i]}", transform=ax.transData, fontsize=11, verticalalignment='top', color=colors[0]) 
    ax.text(t-2, -0.13, f"{nums[1][i]}", transform=ax.transData, fontsize=11, verticalalignment='top', color=colors[1]) 
    ax.text(t-2, -0.19, f"{nums[2][i]}", transform=ax.transData, fontsize=11, verticalalignment='top', color=colors[2]) 
ax.text(-2, -0.01, "No. at risk", transform=ax.transData, fontsize=11, verticalalignment='top', color="black", fontweight='bold') 
ax.hlines(0,-5, months[-1], color="black", linewidth=.5)
ax.set_ylim([-.26,1])
ax.set_xlim([-5,months[-1]])
ax.set_xticks(range(0,months[-1]+10,10))
ax.set_xticklabels(range(0,months[-1]+10,10))
ax.set_yticks([0,0.2,0.4,0.6,0.8,1])
ax.set_yticklabels([0,0.2,0.4,0.6,0.8,1])
ax.spines['left'].set_bounds(0,1)
ax.spines['bottom'].set_bounds(0,months[-1])
ax.set_xlabel("Time (months)", fontsize=12)
ax.set_ylabel("Overall survival", fontsize=12)
ax.spines[["top", "right"]].set_visible(False)
ax.legend(loc="upper right")

fig.tight_layout()
fig.savefig(os.path.join(figs_folder_UPENN, f"Clustered-groups_Survival-times.{fmt}"), dpi=dpi, format=fmt)

G_STATS_array = np.array(G_STATS)
vals = []
for group in [1,2,3]:
    chisquared, p_val, stats, covariance = compare_survival(STATS[G_STATS_array!=group], G_STATS_array[G_STATS_array!=group], return_stats=True)
    print(f"Ommited cluster {group-1}: Statistic = {chisquared} and p-value = {p_val}")
    vals.append(p_val)

_, corrected = fdrcorrection(vals, alpha=0.05, method='p', is_sorted=False)
print("FDR corrected results: ", corrected)

#### Joint cohorts

In [ ]:
kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
kmeans.fit_predict(DATA[["volume", "tdi"]])

# Create subplots
fig, ax = plt.subplots(2, 2, figsize=(8,8))

# Scatter plot for age vs ltdi
ax[0,0].scatter(DATA["age"], DATA["ltdi"], s=10, color="tab:gray")
ax[0,0].spines[["top","right","bottom"]].set_visible(False)
ax[0,0].set_xlim([15,100])
ax[0,0].set_ylim([0,15])
ax[0,0].set_xticks([])
ax[0,0].set_ylabel("Lesion-tract density index", fontsize=12)
corr, pval = pearsonr(DATA["age"], DATA["ltdi"])
ax[0,0].text(
    0.05, 0.9, 
    r'$\rho$'+f" = {corr:.2f}\np = {pval:.3f}", transform=ax[0,0].transAxes, verticalalignment='top', bbox=dict(boxstyle="round", alpha=0.1), 
    color='red' if pval < 0.05 else 'black', 
    fontsize=12
)

# Scatter plot for age vs tdi
ax[1,0].scatter(DATA["age"], DATA["tdi"], s=10, color="tab:gray")
ax[1,0].spines[["top","right"]].set_visible(False)
ax[1,0].set_xlim([15,100])
ax[1,0].set_ylim([0,300])
ax[1,0].set_ylabel("Tract density index", fontsize=12)
ax[1,0].set_xlabel("Age at MRI (years)", fontsize=12)
corr, pval = pearsonr(DATA["age"], DATA["tdi"])
ax[1,0].text(
    0.05, 0.9, 
    r'$\rho$'+f" = {corr:.2f}\np = {pval:.3f}", transform=ax[1,0].transAxes, verticalalignment='top', bbox=dict(boxstyle="round", alpha=0.1), 
    color='red' if pval < 0.05 else 'black', 
    fontsize=12
)

# Scatter plot for volume vs ltdi
ax[0,1].scatter(DATA["volume"], DATA["ltdi"], s=10, color="tab:gray")
ax[0,1].spines[["top","right","bottom"]].set_visible(False)
ax[0,1].set_xlim([0,320])
ax[0,1].set_ylim([0,15])
ax[0,1].set_xticks([])
ax[0,1].set_ylabel("Lesion-tract density index", fontsize=12)
corr, pval = pearsonr(DATA["volume"], DATA["ltdi"])
ax[0,1].text(
    0.05, 0.9, 
    r'$\rho$'+f" = {corr:.2f}\np = {pval:.3f}", transform=ax[0,1].transAxes, verticalalignment='top', bbox=dict(boxstyle="round", alpha=0.1), 
    color='red' if pval < 0.05 else 'black', 
    fontsize=12
)

# Scatter plot for volume vs tdi
colors = ['#1b9e77', '#d95f02', '#7570b3']  # Teal, Orange, Purple
ax[1,1].scatter(DATA["volume"], DATA["tdi"], s=10, c=[colors[label] for label in kmeans.labels_], marker='o', alpha=0.25)
for i in range(kmeans.n_clusters):
    ax[1,1].scatter(kmeans.cluster_centers_[i, 0], kmeans.cluster_centers_[i, 1], c=colors[i], marker='x', s=100, linewidth=3)
ax[1,1].spines[["top","right"]].set_visible(False)
ax[1,1].set_xlim([0,320])
ax[1,1].set_ylim([0,300])
ax[1,1].set_xlabel("Tumor volume ("+r'$cm^3$'+")", fontsize=12)
ax[1,1].set_ylabel("Tract density index", fontsize=12)
corr, pval = pearsonr(DATA["volume"], DATA["tdi"])
ax[1,1].text(
    0.70, 0.9, 
    r'$\rho$'+f" = {corr:.2f}\np = {pval:.3f}", 
    transform=ax[1,1].transAxes, verticalalignment='top', bbox=dict(boxstyle="round", alpha=0.1), 
    color='red' if pval < 0.05 else 'black', 
    fontsize=12
)

# Title and layout
fig.suptitle(f"Joint Cohorts (N={all_upenn+all_ucsf})", fontweight="bold")
fig.tight_layout()
fig.savefig(os.path.join(figs_folder_Joint, f"Scatter-distributions_Clusters-VlmTDI_Joint-cohort.{fmt}"), dpi=dpi, format=fmt)

In [ ]:
os.makedirs(os.path.join(figs_folder_Joint,"NIFTIs"), exist_ok=True)

In [ ]:
# Execitopm time around 0.5 mins if the results can be loaded
# Execution time around 15-17 mins if the data needs to be computed from scratch
tumor_masks = glob.glob(f"{root_upenn}/TDMaps_IDH1-WT/*/masks/*-whole*") + glob.glob(f"{root_ucsf}/TDMaps_Grade-IV/*/masks/*-whole*")

small_small = DATA["ID"].values[kmeans.labels_==2]
small_big = DATA["ID"].values[kmeans.labels_==1]
big_medium = DATA["ID"].values[kmeans.labels_==0]

try:
    print("Loading pre-existing results ...")
    joint_masks_ss = nib.load(os.path.join(figs_folder_Joint, "NIFTIs", "Lesions__Cluster-2_Vlm-small_TDI-small.nii.gz")).get_fdata()
    joint_masks_sb = nib.load(os.path.join(figs_folder_Joint, "NIFTIs", "Lesions__Cluster-1_Vlm-small_TDI-big.nii.gz")).get_fdata()
    joint_masks_bm = nib.load(os.path.join(figs_folder_Joint, "NIFTIs", "Lesions__Cluster-0_Vlm-big_TDI-medium.nii.gz")).get_fdata()
    print("Done!")
except:
    print("Loading data failed! Creating the samples from the data")
    Nss, Nsb, Nbm = 0, 0, 0
    for file in tqdm(tumor_masks):
        if "UCSF" in file:
            id = file.split("_")[-2][-14:]
            id = rewrite_subjectID(id)
        elif "UPENN" in file:
            id = file.split("/")[-1].split(".")[0].split("tissue")[0][-19:-1]
        else:
            print("Error with", file)
        
        if id in small_small:
            if Nss == 0:
                joint_masks_ss = nib.load(file).get_fdata()
            else:
                joint_masks_ss += nib.load(file).get_fdata()
            Nss += 1
        
        if id in small_big:
            if Nsb == 0:
                joint_masks_sb = nib.load(file).get_fdata()
            else:
                joint_masks_sb += nib.load(file).get_fdata()
            Nsb += 1        
            
        if id in big_medium:
            if Nbm == 0:
                joint_masks_bm = nib.load(file).get_fdata()
            else:
                joint_masks_bm += nib.load(file).get_fdata()
            Nbm += 1

    print("--------------------------------------")
    print("Saving files ...")
    nib.save(nib.Nifti1Image(joint_masks_ss, T1.affine, T1.header), os.path.join(figs_folder_Joint, "NIFTIs", "Lesions__Cluster-2_Vlm-small_TDI-small.nii.gz"))
    nib.save(nib.Nifti1Image(joint_masks_bm, T1.affine, T1.header), os.path.join(figs_folder_Joint, "NIFTIs", "Lesions__Cluster-0_Vlm-big_TDI-medium.nii.gz"))
    nib.save(nib.Nifti1Image(joint_masks_sb, T1.affine, T1.header), os.path.join(figs_folder_Joint, "NIFTIs", "Lesions__Cluster-1_Vlm-small_TDI-big.nii.gz"))

    print(f"Number of samples in clsuter 0: {Nbm}")
    print(f"Number of samples in clsuter 1: {Nsb}")
    print(f"Number of samples in clsuter 2: {Nss}")
    print("--------------------------------------")
    print(f"Total number of samples: {Nss+Nsb+Nbm}")

In [ ]:
# Execitopm time around 10-12 mins if the results can be loaded
# Execution time around 35-40 mins if the data needs to be computed from scratch
ltdis = glob.glob(f"{root_upenn}/TDMaps_IDH1-WT/*/maps/*-whole_TDMap-lesion*") + glob.glob(f"{root_ucsf}/TDMaps_Grade-IV/*/maps/*-whole_TDMap-lesion*")

# We reiterate the definition of the groups just to be sure
small_small = DATA["ID"].values[kmeans.labels_==2]
small_big = DATA["ID"].values[kmeans.labels_==1]
big_medium = DATA["ID"].values[kmeans.labels_==0]

try:
    print("Loading pre-existing results ...")
    joint_Ltracts_ss = nib.load(os.path.join(figs_folder_Joint, "NIFTIs", "Ltracts__Cluster-2_Vlm-small_TDI-small.nii.gz")).get_fdata()
    joint_Ltracts_sb = nib.load(os.path.join(figs_folder_Joint, "NIFTIs", "Ltracts__Cluster-1_Vlm-small_TDI-big.nii.gz")).get_fdata()
    joint_Ltracts_bm = nib.load(os.path.join(figs_folder_Joint, "NIFTIs", "Ltracts__Cluster-0_Vlm-big_TDI-medium.nii.gz")).get_fdata()
    Nss, Nsb, Nbm = len(joint_Ltracts_ss), len(joint_Ltracts_sb), len(joint_Ltracts_bm)joint

    joint_Ltracts_ss_mean = nib.load(os.path.join(figs_folder_Joint, "NIFTIs", "Ltracts-mean__Cluster-2_Vlm-small_TDI-small.nii.gz")).get_fdata()
    joint_Ltracts_sb_mean = nib.load(os.path.join(figs_folder_Joint, "NIFTIs", "Ltracts-mean__Cluster-1_Vlm-small_TDI-big.nii.gz")).get_fdata()
    joint_Ltracts_bm_mean = nib.load(os.path.join(figs_folder_Joint, "NIFTIs", "Ltracts-mean__Cluster-0_Vlm-big_TDI-medium.nii.gz")).get_fdata()

    joint_Ltracts_ss_std = nib.load(os.path.join(figs_folder_Joint, "NIFTIs", "Ltracts-std__Cluster-2_Vlm-small_TDI-small.nii.gz")).get_fdata()
    joint_Ltracts_sb_std = nib.load(os.path.join(figs_folder_Joint, "NIFTIs", "Ltracts-std__Cluster-1_Vlm-small_TDI-big.nii.gz")).get_fdata()
    joint_Ltracts_bm_std = nib.load(os.path.join(figs_folder_Joint, "NIFTIs", "Ltracts-std__Cluster-0_Vlm-big_TDI-medium.nii.gz")).get_fdata()
except:
    print("Loading data failed! Creating the samples from the data")
    Nss, Nsb, Nbm = 0, 0, 0
    joint_Ltracts_ss, joint_Ltracts_sb, joint_Ltracts_bm = [], [], []
    for file in tqdm(ltdis):
        if "UCSF" in file:
            id = file.split("_")[-3][-14:]
            id = rewrite_subjectID(id)
        elif "UPENN" in file:
            id = file.split("/")[-1].split(".")[0].split("tissue")[0][-19:-1]
        else:
            print("Error with", file)

        ldata = nib.load(file).get_fdata()
        if id in small_small:
            joint_Ltracts_ss.append(ldata)
            Nss += 1
        
        if id in small_big:
            joint_Ltracts_sb.append(ldata)
            Nsb += 1        
            
        if id in big_medium:
            joint_Ltracts_bm.append(ldata)
            Nbm += 1

    print("--------------------------------------")
    print("Computing the mean and std tract density per voxel ...")
    joint_Ltracts_ss, joint_Ltracts_sb, joint_Ltracts_bm = np.array(joint_Ltracts_ss), np.array(joint_Ltracts_sb), np.array(joint_Ltracts_bm)
    joint_Ltracts_ss_mean, joint_Ltracts_sb_mean, joint_Ltracts_bm_mean = joint_Ltracts_ss.mean(axis=0), joint_Ltracts_sb.mean(axis=0), joint_Ltracts_bm.mean(axis=0)
    joint_Ltracts_ss_std, joint_Ltracts_sb_std, joint_Ltracts_bm_std = joint_Ltracts_ss.std(axis=0), joint_Ltracts_sb.std(axis=0), joint_Ltracts_bm.std(axis=0)

    print("--------------------------------------")
    print("Saving files ...")
    nib.save(nib.Nifti1Image(joint_Ltracts_ss, T1.affine, T1.header), os.path.join(figs_folder_Joint, "NIFTIs", "Ltracts__Cluster-2_Vlm-small_TDI-small.nii.gz"))
    nib.save(nib.Nifti1Image(joint_Ltracts_ss_mean, T1.affine, T1.header), os.path.join(figs_folder_Joint, "NIFTIs" "Ltracts-mean__Cluster-2_Vlm-small_TDI-small.nii.gz"))
    nib.save(nib.Nifti1Image(joint_Ltracts_ss_std, T1.affine, T1.header), os.path.join(figs_folder_Joint, "NIFTIs", "Ltracts-std__Cluster-2_Vlm-small_TDI-small.nii.gz"))

    nib.save(nib.Nifti1Image(joint_Ltracts_bm, T1.affine, T1.header), os.path.join(figs_folder_Joint, "NIFTIs", "Ltracts__Cluster-0_Vlm-big_TDI-medium.nii.gz"))
    nib.save(nib.Nifti1Image(joint_Ltracts_bm_mean, T1.affine, T1.header), os.path.join(figs_folder_Joint, "NIFTIs", "Ltracts-mean__Cluster-0_Vlm-big_TDI-medium.nii.gz"))
    nib.save(nib.Nifti1Image(joint_Ltracts_bm_std, T1.affine, T1.header), os.path.join(figs_folder_Joint, "NIFTIs", "Ltracts-std__Cluster-0_Vlm-big_TDI-medium.nii.gz"))

    nib.save(nib.Nifti1Image(joint_Ltracts_sb, T1.affine, T1.header), os.path.join(figs_folder_Joint, "NIFTIs", "Ltracts__Cluster-1_Vlm-small_TDI-big.nii.gz"))
    nib.save(nib.Nifti1Image(joint_Ltracts_sb_mean, T1.affine, T1.header), os.path.join(figs_folder_Joint, "NIFTIs", "Ltracts-mean__Cluster-1_Vlm-small_TDI-big.nii.gz"))
    nib.save(nib.Nifti1Image(joint_Ltracts_sb_std, T1.affine, T1.header), os.path.join(figs_folder_Joint, "NIFTIs", "Ltracts-std__Cluster-1_Vlm-small_TDI-big.nii.gz"))

print("--------------------------------------")
print(f"Number of samples in clsuter 0 (big-medium): {Nbm}")
print(f"Number of samples in clsuter 1 (small-big): {Nsb}")
print(f"Number of samples in clsuter 2 (small-small): {Nss}")
print("======================================")
print(f"Total number of samples: {Nss+Nsb+Nbm}")

In [ ]:
# The NIFTI images are saved (and can be loaded instead of computed everytime). These two thresholds are used to plot the maps in pdf format for rapid inspection
mask_threshold = 10
tract_threshold = 15
std_threshold = 10

plot_lesions(joint_masks_ss, T1, save_figname=os.path.join(figs_folder_Joint, "Lesions__Cluster-2_Vlm-small_TDI-small.pdf"), title=None, color_map="Purples", threshold=mask_threshold, Nslices=15, dpi=400)
plot_lesions(joint_Ltracts_ss_mean, T1, save_figname=os.path.join(figs_folder_Joint, "Ltracts-mean__Cluster-2_Vlm-small_TDI-small.pdf"), title=None, color_map="Purples", threshold=tract_threshold, Nslices=15, dpi=400)
plot_lesions(joint_Ltracts_ss_std, T1, save_figname=os.path.join(figs_folder_Joint, "Ltracts-std__Cluster-2_Vlm-small_TDI-small.pdf"), title=None, color_map="viridis", threshold=std_threshold, Nslices=15, dpi=400)

In [ ]:
# The NIFTI images are saved (and can be loaded instead of computed everytime). These two thresholds are used to plot the maps in pdf format for rapid inspection
mask_threshold = 30
tract_threshold = 20
std_threshold = 10

plot_lesions(joint_masks_sb, T1, save_figname=os.path.join(figs_folder_Joint, "Lesions__Cluster-1_Vlm-small_TDI-big.pdf"), title=None, color_map="Oranges", threshold=mask_threshold, Nslices=15, dpi=400)
plot_lesions(joint_Ltracts_sb_mean, T1, save_figname=os.path.join(figs_folder_Joint, "Ltracts-mean__Cluster-1_Vlm-small_TDI-big.pdf"), title=None, color_map="Oranges", threshold=tract_threshold, Nslices=15, dpi=400)
plot_lesions(joint_Ltracts_sb_std, T1, save_figname=os.path.join(figs_folder_Joint, "Ltracts-std__Cluster-1_Vlm-small_TDI-big.pdf"), title=None, color_map="viridis", threshold=std_threshold, Nslices=15, dpi=400)

In [ ]:
# The NIFTI images are saved (and can be loaded instead of computed everytime). These two thresholds are used to plot the maps in pdf format for rapid inspection
mask_threshold = 30
tract_threshold = 20
std_threshold = 10

plot_lesions(joint_masks_bm, T1, save_figname=os.path.join(figs_folder_Joint, "Lesions__Cluster-0_Vlm-big_TDI-medium.pdf"), title=None, color_map="BuGn", threshold=mask_threshold, Nslices=15, dpi=400)
plot_lesions(joint_Ltracts_bm_mean, T1, save_figname=os.path.join(figs_folder_Joint, "Ltracts-mean__Cluster-0_Vlm-big_TDI-medium.pdf"), title=None, color_map="BuGn", threshold=tract_threshold, Nslices=15, dpi=400)
plot_lesions(joint_Ltracts_bm_std, T1, save_figname=os.path.join(figs_folder_Joint, "Ltracts-std__Cluster-0_Vlm-big_TDI-medium.pdf"), title=None, color_map="viridis", threshold=std_threshold, Nslices=15, dpi=400)

In [ ]:
# The NIFTI images are saved (and can be loaded instead of computed everytime). These two thresholds are used to plot the maps in pdf format for rapid inspection
ov_tract_threshold = 20
bin_threshold = 0

plot_lesions(
    np.where(joint_Ltracts_ss>bin_threshold, 1, 0).sum(axis=0), 
    T1, 
    save_figname=os.path.join(figs_folder_Joint, "Ltracts__Cluster-2_Vlm-small_TDI-small.pdf"), 
    title=None, 
    color_map="Purples", 
    threshold=ov_tract_threshold, 
    Nslices=15, 
    dpi=400
)
plot_lesions(
    np.where(joint_Ltracts_sb>bin_threshold, 1, 0).sum(axis=0), 
    T1, 
    save_figname=os.path.join(figs_folder_Joint, "Ltracts__Cluster-1_Vlm-small_TDI-big.pdf"), 
    title=None, 
    color_map="Oranges", 
    threshold=ov_tract_threshold, 
    Nslices=15, 
    dpi=400
)
plot_lesions(
    np.where(joint_Ltracts_bm>bin_threshold, 1, 0).sum(axis=0), 
    T1, 
    save_figname=os.path.join(figs_folder_Joint, "Ltracts__Cluster-0_Vlm-big_TDI-medium.pdf"), 
    title=None, 
    color_map="BuGn", 
    threshold=ov_tract_threshold, 
    Nslices=15, 
    dpi=400
)

In [ ]:
# We have the data in a way to perform statistical testing. 
# Not with the mean and standard deviations (althought we could compute a z-score)
# But with the 3 different sample populations
# Since we operate on the voxel level, we have to be careful to correct for multiple comparissons

# TODO: Think!

In [ ]:
months = range(0,110,10)
colors = ['#1b9e77', '#d95f02', '#7570b3'] # Teal, Orange, Purple
labels = ['Large volume & Medium TDI', 'Small volume & Large TDI', 'Small volume & Small TDI']

fig, ax = plt.subplots(1,1, figsize=(6,5))
STATS = []
G_STATS = []
nums = np.empty(shape=(3,), dtype=object)
for i, D in enumerate(np.unique(kmeans.labels_)):
    diag = DATA[kmeans.labels_==D]
    mask = ~np.isnan(diag["status"]) & ~np.isnan(diag["OS"])
    diag = diag[mask]
    nums[i] = []
    for t in months:
        nums[i].append((diag["OS"]>=(t*daysXmonth)).sum())
    
    time, survival_prob, conf_int = kaplan_meier_estimator(
                diag["status"]==1, diag["OS"], conf_type="log-log"
            )
    ax.step(time/daysXmonth, survival_prob, where="post", label=f"{labels[i]} (Cluster {D})", color=colors[i], alpha=0.75)
    ax.fill_between(time/daysXmonth, conf_int[0], conf_int[1], alpha=0.15, step="post", color=colors[i])
    for t in diag.loc[diag["status"]==0, "OS"].values: # Censoring times
        ax.plot(time[time==t]/daysXmonth, survival_prob[time==t], "|", color=colors[i])

    STATS.extend([(st, ovs) for st, ovs in zip(diag["status"]==1, diag["OS"].values)])
    G_STATS.extend([i+1 for ovs in diag["OS"].values])

STATS = np.array(STATS, dtype=[('event', 'bool'),('time', 'float')])
chisquared, p_val, stats, covariance = compare_survival(STATS, G_STATS, return_stats=True)
ax.text(0.70, 0.60, r"$\chi^2 =$"+f"{round(chisquared,4)}, \np = {round(p_val,4)}", transform=ax.transAxes, 
                    fontsize=10, verticalalignment='top', bbox=dict(boxstyle="round", alpha=0.1), color="red" if p_val<=0.05 else "black")    

for i,t in enumerate(months):
    ax.text(t-2, -0.07, f"{nums[0][i]}", transform=ax.transData, fontsize=11, verticalalignment='top', color=colors[0]) 
    ax.text(t-2, -0.13, f"{nums[1][i]}", transform=ax.transData, fontsize=11, verticalalignment='top', color=colors[1]) 
    ax.text(t-2, -0.19, f"{nums[2][i]}", transform=ax.transData, fontsize=11, verticalalignment='top', color=colors[2]) 
ax.text(-2, -0.01, "No. at risk", transform=ax.transData, fontsize=11, verticalalignment='top', color="black", fontweight='bold') 
ax.hlines(0,-5, months[-1], color="black", linewidth=.5)
ax.set_ylim([-.26,1])
ax.set_xlim([-5,months[-1]])
ax.set_xticks(range(0,months[-1]+10,10))
ax.set_xticklabels(range(0,months[-1]+10,10))
ax.set_yticks([0,0.2,0.4,0.6,0.8,1])
ax.set_yticklabels([0,0.2,0.4,0.6,0.8,1])
ax.spines['left'].set_bounds(0,1)
ax.spines['bottom'].set_bounds(0,months[-1])
ax.set_xlabel("Time (months)", fontsize=12)
ax.set_ylabel("Overall survival", fontsize=12)
ax.spines[["top", "right"]].set_visible(False)
ax.legend()

fig.tight_layout()
fig.savefig(os.path.join(figs_folder_Joint, f"Clustered-groups_Survival-times.{fmt}"), dpi=dpi, format=fmt)

G_STATS_array = np.array(G_STATS)
vals = []
for group in [1,2,3]:
    chisquared, p_val, stats, covariance = compare_survival(STATS[G_STATS_array!=group], G_STATS_array[G_STATS_array!=group], return_stats=True)
    print(f"Ommited cluster {group-1}: Statistic = {chisquared} and p-value = {p_val}")
    vals.append(p_val)

_, corrected = fdrcorrection(vals, alpha=0.05, method='p', is_sorted=False)
print("FDR corrected results: ", corrected)

In [ ]:
colors = ['#1b9e77', '#d95f02', '#7570b3']
labels = ['Large volume & Medium TDI', 'Small volume & Large TDI', 'Small volume & Small TDI']
fig, ax = plt.subplots(1,1, figsize=(6,5))
STATS = []
G_STATS = []
nums = np.empty(shape=(3,), dtype=object)
for i, D in enumerate(np.unique(kmeans.labels_)):
    diag = DATA[kmeans.labels_==D]
    mask = ~np.isnan(diag["status"]) & ~np.isnan(diag["OS"])
    diag = diag[mask]
    diag["OS"] = diag["OS"] * np.exp(Cmodel.coef_[0]*cohort)
    nums[i] = []
    for t in months:
        nums[i].append((diag["OS"]>=(t*daysXmonth)).sum())
    
    time, survival_prob, conf_int = kaplan_meier_estimator(
                diag["status"]==1, diag["OS"], conf_type="log-log"
            )
    ax.step(time/daysXmonth, survival_prob, where="post", label=f"{labels[i]} (Cluster {D})", color=colors[i], alpha=0.75)
    ax.fill_between(time/daysXmonth, conf_int[0], conf_int[1], alpha=0.15, step="post", color=colors[i])
    for t in diag.loc[diag["status"]==0, "OS"].values: # Censoring times
        ax.plot(time[time==t]/daysXmonth, survival_prob[time==t], "|", color=colors[i])

    STATS.extend([(st, ovs) for st, ovs in zip(diag["status"]==1, diag["OS"].values)])
    G_STATS.extend([i+1 for ovs in diag["OS"].values])

STATS = np.array(STATS, dtype=[('event', 'bool'),('time', 'float')])
chisquared, p_val, stats, covariance = compare_survival(STATS, G_STATS, return_stats=True)
ax.text(0.70, 0.60, r"$\chi^2 =$"+f"{round(chisquared,4)}, \np = {round(p_val,4)}", transform=ax.transAxes, 
                    fontsize=10, verticalalignment='top', bbox=dict(boxstyle="round", alpha=0.1), color="red" if p_val<=0.05 else "black")    

for i,t in enumerate(months):
    ax.text(t-2, -0.07, f"{nums[0][i]}", transform=ax.transData, fontsize=11, verticalalignment='top', color=colors[0]) 
    ax.text(t-2, -0.13, f"{nums[1][i]}", transform=ax.transData, fontsize=11, verticalalignment='top', color=colors[1]) 
    ax.text(t-2, -0.19, f"{nums[2][i]}", transform=ax.transData, fontsize=11, verticalalignment='top', color=colors[2]) 
ax.text(-2, -0.01, "No. at risk", transform=ax.transData, fontsize=11, verticalalignment='top', color="black", fontweight='bold') 
ax.hlines(0,-5, months[-1], color="black", linewidth=.5)
ax.set_ylim([-.26,1])
ax.set_xlim([-5,months[-1]])
ax.set_xticks(range(0,months[-1]+10,10))
ax.set_xticklabels(range(0,months[-1]+10,10))
ax.set_yticks([0,0.2,0.4,0.6,0.8,1])
ax.set_yticklabels([0,0.2,0.4,0.6,0.8,1])
ax.spines['left'].set_bounds(0,1)
ax.spines['bottom'].set_bounds(0,months[-1])
ax.set_xlabel("Time (months)", fontsize=12)
ax.set_ylabel("Overall survival", fontsize=12)
ax.spines[["top", "right"]].set_visible(False)
ax.legend()

fig.tight_layout()
fig.savefig(os.path.join(figs_folder_Joint, f"Clustered-groups_Survival-times_Site-effectCorrected.{fmt}"), dpi=dpi, format=fmt)

G_STATS_array = np.array(G_STATS)
vals = []
for group in [1,2,3]:
    chisquared, p_val, stats, covariance = compare_survival(STATS[G_STATS_array!=group], G_STATS_array[G_STATS_array!=group], return_stats=True)
    print(f"Ommited cluster {group-1}: Statistic = {chisquared} and p-value = {p_val}")
    vals.append(p_val)

_, corrected = fdrcorrection(vals, alpha=0.05, method='p', is_sorted=False)
print("FDR corrected results: ", corrected)